**Azure Open AI Endpoint and Key:  GPT - 4o - Mini Model**

In [ ]:
import os
from openai import AzureOpenAI

endpoint = "https://areypragir-4130-gpt4omi-resource.cognitiveservices.azure.com/"
model_name = "gpt-4o-mini"
deployment = "gpt-4o-mini"

subscription_key = "##"
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "I am going to Paris, what should I see?",
        }
    ],
    max_tokens=4096,
    temperature=1.0,
    top_p=1.0,
    model=deployment
)

print(response.choices[0].message.content)

Paris is a vibrant city filled with iconic landmarks, cultural experiences, and picturesque neighborhoods. Here’s a list of must-see attractions and activities during your visit:

### Iconic Landmarks
1. **Eiffel Tower**: Visit this iconic symbol of Paris, and don’t forget to go up for stunning views of the city.
2. **Louvre Museum**: Home to thousands of artworks, including the Mona Lisa and the Venus de Milo.
3. **Notre-Dame Cathedral**: Although under restoration, the facade is still impressive; you can also explore the Île de la Cité.
4. **Sacré-Cœur Basilica**: Located on Montmartre Hill, enjoy the beautiful architecture and panoramic views. 
5. **Arc de Triomphe**: Climb to the top for a great view of the Champs-Élysées.

### Museums and Galleries
6. **Musée d'Orsay**: Famous for its collection of Impressionist and Post-Impressionist masterpieces.
7. **Centre Pompidou**: A modern art museum with a striking architectural design.
8. **Musée de l'Orangerie**: Known for Monet's Water

In [ ]:
pip install ir_datasets tqdm

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.1/866.1 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.1 MB/s eta 0:00:00
  Created wheel for warc3-wet-clueweb09: filename=warc3_wet_clueweb09-0.2.5-py3-none-any.whl size=18919 sha256=62531b6d80204e358011c2fc0682dc666d9453e5afc11447dfd2e11055201010
  Stored in directory: /root/.cache/pip/wheels/f6/85/c2/9f0f621def52a1d5db7d29984f81e45f9fb6dfeb1a4eb6e31c
  Created wheel for cbor: filename=cbor-1.0.0-cp312-cp312-linux_x86_64.whl size=55023 sha256=6f625c716e1c35c035c31a63a991c0c8ec5f311fd16ad39ed876c694d38e6773
  Stored in directory: /root/.cache/pip/wheels/44/3e/21/a739cbcc331a1ab45c326d6edbdac6118de4402f6076e30ff1
Successfully built warc3-wet-clueweb09 cbor


**Collecting Base Query Documents from msmarco and beir**

In [ ]:
"""
FINAL Step A — Build base_dataset_questq.jsonl
Compatible with your available BEIR datasets.
Collects query–document pairs for multi-attribute dataset building.
"""

import ir_datasets, random, json
from tqdm import tqdm

# ✅ Choose only available dataset identifiers
SAMPLES = {
    "msmarco-passage/train": 400,            # open-domain search
    "beir/cqadupstack/programmers": 100,     # StackOverflow-style Q&A
    "beir/fever": 100,                       # fact verification
    "beir/scidocs": 50,                      # academic domain
    "beir/quora": 50,                        # question paraphrasing
}

random.seed(42)

def sample_dataset(name, sample_size):
    print(f"\n📘 Processing {name}")
    ds = ir_datasets.load(name)

    # Ensure dataset has the necessary parts
    for needed in ["queries_iter", "docs_iter", "qrels_iter"]:
        if not hasattr(ds, needed):
            raise AttributeError(f"Dataset {name} missing: {needed}")

    queries = {q.query_id: q.text for q in ds.queries_iter()}
    docs = {d.doc_id: d.text for d in ds.docs_iter()}
    qrels = list(ds.qrels_iter())

    # Build mapping query → list of relevant doc IDs
    pos_map = {}
    for q in qrels:
        if q.relevance > 0:
            pos_map.setdefault(q.query_id, []).append(q.doc_id)

    valid_qids = list(pos_map.keys())
    if not valid_qids:
        print(f"⚠️ No valid qrels for {name}")
        return []

    chosen_qids = random.sample(valid_qids, min(sample_size, len(valid_qids)))
    results = []

    for qid in tqdm(chosen_qids):
        query = queries.get(qid, "")
        pos_docs = pos_map.get(qid, [])
        for did in pos_docs[:3]:  # take up to 3 positive docs
            if did in docs:
                results.append({
                    "dataset": name,
                    "query_id": qid,
                    "query": query,
                    "document": docs[did],
                    "relevance": 1
                })

    print(f"✅ Collected {len(results)} pairs from {name}")
    return results


# ------------------ MAIN EXECUTION ------------------
all_data = []
for name, n in SAMPLES.items():
    try:
        all_data.extend(sample_dataset(name, n))
    except Exception as e:
        print(f"⚠️ Skipping {name}: {e}")

# Save the combined dataset
with open("1_base_dataset_questq.jsonl", "w", encoding="utf8") as f:
    for item in all_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"\n🎯 Done! Collected {len(all_data)} total query–doc pairs.")
print("📂 Output saved to base_dataset_questq.jsonl")


**Adding Attributes to Base Dataset**

In [ ]:
import json, time, random
from tqdm import tqdm
from openai import AzureOpenAI

# 🔹 Azure setup
endpoint = "https://areypragir-4130-gpt4omi-resource.cognitiveservices.azure.com/"
api_version = "2024-12-01-preview"
deployment = "gpt-4o-mini"
api_key = "#hidden"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=api_key
)

# 🔹 Load base dataset
with open("1_base_dataset_questq.jsonl") as f:
    base_data = [json.loads(l) for l in f]

# 🔹 Few-shot examples to guide GPT
FEWSHOT = """
Examples:
1️⃣ Query: "what is the purpose of DNA replication"
Document: "DNA replication ensures each cell gets an exact copy of the DNA during cell division."
Attributes:
{
 "audience": "Student",
 "keyword": ["Biology", "Genetics"],
 "format": "Academic Paper",
 "language": "English",
 "length": "Short",
 "source": "Wikipedia"
}

2️⃣ Query: "price of a bushel of wheat"
Document: "Interactive chart of historical daily wheat prices..."
Attributes:
{
 "audience": "Researcher",
 "keyword": ["Economics", "Agriculture"],
 "format": "Report",
 "language": "English",
 "length": "Short",
 "source": "NewsSite"
}
"""

# 🔹 Attribute generation function
def get_attributes(query, document, retries=2):
    prompt = f"""
You are labeling information retrieval data using InfoSearch-style attributes.

{FEWSHOT}

Now label this new pair.
Return ONLY a valid JSON dictionary (no explanations, no markdown).

Query: {query}
Document: {document[:800]}
JSON:
"""
    for _ in range(retries):
        try:
            resp = client.chat.completions.create(
                model=deployment,
                temperature=0.4,
                max_tokens=250,
                messages=[{"role": "user", "content": prompt}]
            )
            content = resp.choices[0].message.content.strip()
            # Clean and try parsing JSON
            start = content.find("{")
            end = content.rfind("}") + 1
            json_part = content[start:end]
            attrs = json.loads(json_part)
            # Ensure all expected keys exist
            required = {"audience", "keyword", "format", "language", "length", "source"}
            if required.issubset(attrs.keys()):
                return attrs
        except Exception as e:
            time.sleep(1)
            continue
    return None

# 🔹 Process all samples
enriched = []
for i, item in enumerate(tqdm(base_data, desc="Annotating")):
    attrs = get_attributes(item["query"], item["document"])
    if attrs:
        item.update(attrs)
        enriched.append(item)

# 🔹 Save
with open("2_base_with_multi_attri.jsonl", "w", encoding="utf8") as f:
    for e in enriched:
        f.write(json.dumps(e, ensure_ascii=False) + "\n")

print(f"✅ Created multi_attr_dataset.jsonl with {len(enriched)} labeled pairs.")


**Multi-Attribute Instructed + Reversed Query Generation**

In [ ]:
#1. Install required packages ---
!pip install openai aiohttp nest_asyncio -q

#2. Imports and setup ---
import json, re, time, asyncio, nest_asyncio
from openai import AsyncAzureOpenAI
nest_asyncio.apply()

#3. Azure OpenAI configuration ---
API_KEY = "#hidden"
ENDPOINT = "https://areypragir-4130-gpt4omi-resource.cognitiveservices.azure.com/"
API_VERSION = "2024-12-01-preview"
MODEL = "gpt-4o-mini"

client = AsyncAzureOpenAI(
    api_key=API_KEY,
    azure_endpoint=ENDPOINT,
    api_version=API_VERSION
)

#4. Rate-limit and save settings ---
TOKENS_PER_MINUTE = 100_000
REQUESTS_PER_MINUTE = 1000
MAX_CONCURRENT_REQUESTS = 8
SLEEP_BETWEEN_REQUESTS = 60 / REQUESTS_PER_MINUTE
SAVE_INTERVAL = 50   # Auto-save every 50 queries

def build_prompt(query, attributes):
    """Prompt asking GPT to return instructed and reversed versions as strict JSON."""
    attr_text = ", ".join([f"{k}: {v}" for k, v in attributes.items() if v])
    return f"""
You are generating search queries with multiple document-level attributes.

Given a base query and its attributes, produce:
1. An instructed version that naturally includes 2–3 attributes.
2. A reversed instructed version that logically negates those same attributes.

Return output ONLY as a JSON object with two keys:
"instructed_query" and "reversed_query".

Example:
Base query: "best travel destinations in Europe"
Attributes: format=blog, language=English, audience=layman

Output:
{{
  "instructed_query": "List the best travel destinations in Europe. Please provide a blog in English for laymen.",
  "reversed_query": "List the best travel destinations in Europe. Please do not provide a blog in English for laymen."
}}

Now for this:
Base query: "{query}"
Attributes: {attr_text}
"""

def safe_json_parse(text):
    """Try robust JSON parsing; fallback to regex extraction if needed."""
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        instructed = re.search(r'"?instructed[_ ]?query"?[:\-]?\s*["“](.*?)["”]', text, re.I | re.S)
        reversed_q = re.search(r'"?reversed[_ ]?query"?[:\-]?\s*["“](.*?)["”]', text, re.I | re.S)
        return {
            "instructed_query": instructed.group(1).strip() if instructed else "",
            "reversed_query": reversed_q.group(1).strip() if reversed_q else ""
        }

# Async GPT-4 call for one entry

async def process_entry(entry):
    query = entry["query"]
    attributes = {
        "audience": entry.get("audience", ""),
        "format": entry.get("format", ""),
        "language": entry.get("language", ""),
        "length": entry.get("length", ""),
        "source": entry.get("source", "")
    }

    prompt = build_prompt(query, attributes)

    try:
        response = await client.chat.completions.create(
            model=MODEL,
            temperature=0.4,
            messages=[{"role": "user", "content": prompt}],
            response_format={"type": "json_object"}  # force JSON output
        )

        content = response.choices[0].message.content.strip()
        parsed = safe_json_parse(content)

        entry["instructed_query"] = parsed.get("instructed_query", "")
        entry["reversed_query"] = parsed.get("reversed_query", "")
        return entry

    except Exception as e:
        print(f"❌ Error on {entry.get('query_id','?')}: {e}")
        return None


# Async batch generator

async def generate_instructed_queries():
    input_file = "2_base_with_multi_attri.jsonl"
    output_file = "multi_attr_instructed.jsonl"

    with open(input_file, "r") as f:
        data = [json.loads(line) for line in f]

    print(f"📘 Loaded {len(data)} entries from {input_file}")

    semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)
    processed = []

    async def limited_process(entry):
        async with semaphore:
            result = await process_entry(entry)
            await asyncio.sleep(SLEEP_BETWEEN_REQUESTS)
            return result

    tasks = [limited_process(e) for e in data]

    for i, coro in enumerate(asyncio.as_completed(tasks), 1):
        result = await coro
        if result:
            processed.append(result)

        # Auto-save every N records
        if i % SAVE_INTERVAL == 0:
            with open(output_file, "a") as f:
                for p in processed[-SAVE_INTERVAL:]:
                    f.write(json.dumps(p, ensure_ascii=False) + "\n")
            print(f"💾 Auto-saved {i} entries...")

    # Final save
    with open(output_file, "a") as f:
        for p in processed:
            f.write(json.dumps(p, ensure_ascii=False) + "\n")

    print(f"✅ Finished generating {len(processed)} instructed queries!")
    print(f"📁 Output: {output_file}")


await generate_instructed_queries()


📘 Loaded 715 entries from multi_attr_dataset.jsonl
💾 Auto-saved 50 entries...
💾 Auto-saved 100 entries...
💾 Auto-saved 150 entries...
💾 Auto-saved 200 entries...
💾 Auto-saved 250 entries...
💾 Auto-saved 300 entries...
💾 Auto-saved 350 entries...
💾 Auto-saved 400 entries...
💾 Auto-saved 450 entries...
💾 Auto-saved 500 entries...
💾 Auto-saved 550 entries...
💾 Auto-saved 600 entries...
💾 Auto-saved 650 entries...
💾 Auto-saved 700 entries...
✅ Finished generating 715 instructed queries!
📁 Output: multi_attr_instructed.jsonl


**Document Rewriting + Hard Negative Generation**

In [ ]:
# ====================================================
# 🚀 Step 4 — Document Rewriting + Hard Negative Generation
# ====================================================

!pip install openai aiohttp nest_asyncio -q

import json, re, time, asyncio, nest_asyncio
from openai import AsyncAzureOpenAI
nest_asyncio.apply()

# --- Azure config (same as Step 3) ---
API_KEY = "##"
ENDPOINT = "https://areypragir-4130-gpt4omi-resource.cognitiveservices.azure.com/"
API_VERSION = "2024-12-01-preview"
MODEL = "gpt-4o-mini"

client = AsyncAzureOpenAI(
    api_key=API_KEY,
    azure_endpoint=ENDPOINT,
    api_version=API_VERSION
)

# --- Rate control ---
MAX_CONCURRENT_REQUESTS = 8
REQUESTS_PER_MINUTE = 1000
SLEEP_BETWEEN_REQUESTS = 60 / REQUESTS_PER_MINUTE
SAVE_INTERVAL = 50


# ====================================================
# 🧠 Prompt builder
# ====================================================

def build_doc_prompt(document, attributes):
    attr_text = ", ".join([f"{k}: {v}" for k, v in attributes.items() if v])
    return f"""
You are refining a document for an information-retrieval benchmark.

Task 1 – Rewrite the base document so that it **fully satisfies all given attributes**.
Task 2 – Create **one hard negative document** that is **topically similar** but **violates at least one attribute**
(e.g., wrong format, language, or audience).

Return **only** a JSON object:
{{
  "positive_doc": "... rewritten version matching attributes ...",
  "hard_negative_doc": "... realistic but violating version ..."
}}

Base document:
\"\"\"{document}\"\"\"
Attributes: {attr_text}
"""

# ====================================================
# ⚙️ Async single entry processor
# ====================================================

def safe_json_parse(text):
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        pos = re.search(r'"?positive[_ ]?doc"?[:\-]?\s*["“](.*?)["”]', text, re.I | re.S)
        neg = re.search(r'"?hard[_ ]?negative[_ ]?doc"?[:\-]?\s*["“](.*?)["”]', text, re.I | re.S)
        return {
            "positive_doc": pos.group(1).strip() if pos else "",
            "hard_negative_doc": neg.group(1).strip() if neg else ""
        }

async def process_doc(entry):
    document = entry.get("document", "")
    attributes = {
        "audience": entry.get("audience", ""),
        "format": entry.get("format", ""),
        "language": entry.get("language", ""),
        "length": entry.get("length", ""),
        "source": entry.get("source", "")
    }

    prompt = build_doc_prompt(document, attributes)
    try:
        response = await client.chat.completions.create(
            model=MODEL,
            temperature=0.5,
            messages=[{"role": "user", "content": prompt}],
            response_format={"type": "json_object"}
        )
        content = response.choices[0].message.content.strip()
        parsed = safe_json_parse(content)
        entry["positive_doc"] = parsed.get("positive_doc", "")
        entry["hard_negative_doc"] = parsed.get("hard_negative_doc", "")
        return entry
    except Exception as e:
        print(f"❌ Error on {entry.get('query_id','?')}: {e}")
        return None


# ====================================================
# 🚀 Async batch processor
# ====================================================

async def rewrite_documents():
    input_file = "multi_attr_instructed.jsonl"
    output_file = "multi_attr_docs.jsonl"

    with open(input_file, "r") as f:
        data = [json.loads(line) for line in f]

    print(f"📘 Loaded {len(data)} entries from {input_file}")
    semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)
    processed = []

    async def limited(entry):
        async with semaphore:
            result = await process_doc(entry)
            await asyncio.sleep(SLEEP_BETWEEN_REQUESTS)
            return result

    tasks = [limited(e) for e in data]

    for i, coro in enumerate(asyncio.as_completed(tasks), 1):
        res = await coro
        if res:
            processed.append(res)
        if i % SAVE_INTERVAL == 0:
            with open(output_file, "a") as f:
                for p in processed[-SAVE_INTERVAL:]:
                    f.write(json.dumps(p, ensure_ascii=False) + "\n")
            print(f"💾 Saved {i} entries...")

    # final save
    with open(output_file, "a") as f:
        for p in processed:
            f.write(json.dumps(p, ensure_ascii=False) + "\n")

    print(f"✅ Finished rewriting {len(processed)} documents!")
    print(f"📁 Output: {output_file}")

# ====================================================
# ▶️ Run rewriting inside Colab/Jupyter
# ====================================================
await rewrite_documents()


📘 Loaded 2818 entries from multi_attr_instructed.jsonl
💾 Saved 50 entries...
❌ Error on 919029: 'NoneType' object has no attribute 'strip'
💾 Saved 100 entries...
💾 Saved 150 entries...
💾 Saved 200 entries...
💾 Saved 250 entries...
💾 Saved 300 entries...
💾 Saved 350 entries...
💾 Saved 400 entries...
💾 Saved 450 entries...
💾 Saved 500 entries...
💾 Saved 550 entries...
💾 Saved 600 entries...
💾 Saved 650 entries...
💾 Saved 700 entries...
💾 Saved 750 entries...
💾 Saved 800 entries...
💾 Saved 850 entries...
💾 Saved 900 entries...
💾 Saved 950 entries...
💾 Saved 1000 entries...
💾 Saved 1050 entries...
💾 Saved 1100 entries...
💾 Saved 1150 entries...
💾 Saved 1200 entries...
💾 Saved 1250 entries...
💾 Saved 1300 entries...
❌ Error on 737282: 'NoneType' object has no attribute 'strip'
💾 Saved 1350 entries...
💾 Saved 1400 entries...
💾 Saved 1450 entries...
💾 Saved 1500 entries...
💾 Saved 1550 entries...
💾 Saved 1600 entries...
💾 Saved 1650 entries...
💾 Saved 1700 entries...
💾 Saved 1750 entries...
💾

**UPDATED WITH ATTRIBUTES MENTIONED**

**Multi-Attribute Instructed + Reversed Query Generation + Document Rewriting + Hard Negative Generation** (Updated)


In [ ]:
# =========================================================
# STEP 1: Install & Imports
# =========================================================
!pip install openai aiohttp nest_asyncio -q

import json, re, time, asyncio, nest_asyncio, random, copy
from openai import AsyncAzureOpenAI
nest_asyncio.apply()

# =========================================================
# STEP 2: Azure OpenAI Configuration
# =========================================================
API_KEY = "##"
ENDPOINT = "https://areypragir-4130-gpt4omi-resource.cognitiveservices.azure.com/"
API_VERSION = "2024-12-01-preview"
MODEL = "gpt-4o-mini"

client = AsyncAzureOpenAI(
    api_key=API_KEY,
    azure_endpoint=ENDPOINT,
    api_version=API_VERSION
)

# =========================================================
# STEP 3: Rate Control
# =========================================================
MAX_CONCURRENT_REQUESTS = 6
REQUESTS_PER_MINUTE = 1000
SLEEP_BETWEEN_REQUESTS = 60 / REQUESTS_PER_MINUTE
SAVE_INTERVAL = 50

# =========================================================
# STEP 4: Helper Functions
# =========================================================

def safe_json_parse(text):
    """Try to parse model output as JSON; fallback to regex."""
    try:
        return json.loads(text)
    except Exception:
        instructed = re.search(r'"?instructed[_ ]?query"?[:\-]?\s*["“](.*?)["”]', text, re.I | re.S)
        reversed_q = re.search(r'"?reversed[_ ]?query"?[:\-]?\s*["“](.*?)["”]', text, re.I | re.S)
        pos = re.search(r'"?positive[_ ]?doc"?[:\-]?\s*["“](.*?)["”]', text, re.I | re.S)
        neg = re.search(r'"?hard[_ ]?negative[_ ]?doc"?[:\-]?\s*["“](.*?)["”]', text, re.I | re.S)
        return {
            "instructed_query": instructed.group(1).strip() if instructed else "",
            "reversed_query": reversed_q.group(1).strip() if reversed_q else "",
            "positive_doc": pos.group(1).strip() if pos else "",
            "hard_negative_doc": neg.group(1).strip() if neg else ""
        }

def build_instruct_prompt(query, attributes):
    """Prompt for Instructed + Reversed query generation."""
    attr_text = ", ".join([f"{k}: {v}" for k, v in attributes.items() if v])
    return f"""
You are generating search queries with multiple document-level attributes.

Given a base query and its attributes, produce:
1. An instructed version that naturally includes all given attributes.
2. A reversed instructed version that logically negates them.

Return a JSON object:
{{
  "instructed_query": "...",
  "reversed_query": "..."
}}

Base query: "{query}"
Attributes: {attr_text}
"""

def build_doc_prompt(document, attributes):
    """Prompt for rewriting + hard negative generation."""
    attr_text = ", ".join([f"{k}: {v}" for k, v in attributes.items() if v])
    return f"""
You are refining a document for an information-retrieval benchmark.

Task 1 – Rewrite the base document so that it **fully satisfies all given attributes**.
Task 2 – Create one **hard negative document** that is **topically similar** but violates one or two attributes.

Return only JSON:
{{
  "positive_doc": "... rewritten version ...",
  "hard_negative_doc": "... violating version ..."
}}

Base document:
\"\"\"{document}\"\"\"
Attributes: {attr_text}
"""

def sample_attr_combinations(entry, num_combinations=5):
    """Generate 3–5 random 2–3 attribute combinations for a single query."""
    all_attrs = {
        "audience": entry.get("audience", ""),
        "format": entry.get("format", ""),
        "language": entry.get("language", ""),
        "length": entry.get("length", ""),
        "source": entry.get("source", "")
    }
    non_empty = {k: v for k, v in all_attrs.items() if v}
    num_combinations = random.randint(3, 5)

    combos = []
    for _ in range(num_combinations):
        chosen = random.sample(list(non_empty.keys()), k=min(random.randint(2, 3), len(non_empty)))
        combos.append({k: non_empty[k] for k in chosen})
    return combos

# =========================================================
# STEP 5: Async Generation
# =========================================================

async def generate_instructed_queries():
    input_file = "2_base_with_multi_attri.jsonl"
    output_file = "3_instructed_reverse_queries_with_attri.jsonl"

    with open(input_file, "r") as f:
        base_data = [json.loads(line) for line in f]

    print(f"📘 Loaded {len(base_data)} core queries")
    semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)
    processed = []

    async def process_combination(base_entry, combo, combo_id):
        """Generate instructed and reversed queries for one combination."""
        new_entry = copy.deepcopy(base_entry)
        new_entry["attributes"] = combo
        new_entry["combo_id"] = combo_id

        prompt = build_instruct_prompt(base_entry["query"], combo)
        try:
            response = await client.chat.completions.create(
                model=MODEL,
                temperature=0.4,
                messages=[{"role": "user", "content": prompt}],
                response_format={"type": "json_object"}
            )
            parsed = safe_json_parse(response.choices[0].message.content)
            new_entry["instructed_query"] = parsed.get("instructed_query", "")
            new_entry["reversed_query"] = parsed.get("reversed_query", "")
            new_entry["query_type"] = "expanded"
            return new_entry
        except Exception as e:
            print(f"❌ Error on {base_entry.get('query_id')}: {e}")
            return None

    tasks = []
    for entry in base_data:
        combos = sample_attr_combinations(entry)
        for i, combo in enumerate(combos):
            tasks.append(process_combination(entry, combo, i+1))

    async def limited_task(task):
        async with semaphore:
            res = await task
            await asyncio.sleep(SLEEP_BETWEEN_REQUESTS)
            return res

    wrapped_tasks = [limited_task(t) for t in tasks]
    for i, coro in enumerate(asyncio.as_completed(wrapped_tasks), 1):
        res = await coro
        if res:
            processed.append(res)
        if i % SAVE_INTERVAL == 0:
            with open(output_file, "a") as f:
                for p in processed[-SAVE_INTERVAL:]:
                    f.write(json.dumps(p, ensure_ascii=False) + "\n")
            print(f"💾 Auto-saved {i} combinations...")

    with open(output_file, "a") as f:
        for p in processed:
            f.write(json.dumps(p, ensure_ascii=False) + "\n")

    print(f"✅ Generated {len(processed)} instructed + reversed combinations!")

# =========================================================
# STEP 6: Document Rewriting
# =========================================================

async def rewrite_documents():
    input_file = "3_instructed_reverse_queries_with_attri.jsonl"
    output_file = "4_rewritten_docs_hard_negatives_with_attri.jsonl"

    with open(input_file, "r") as f:
        data = [json.loads(line) for line in f]

    semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)
    processed = []

    async def process_doc(entry):
        attributes = entry.get("attributes", {})
        prompt = build_doc_prompt(entry.get("document", ""), attributes)
        try:
            response = await client.chat.completions.create(
                model=MODEL,
                temperature=0.5,
                messages=[{"role": "user", "content": prompt}],
                response_format={"type": "json_object"}
            )
            parsed = safe_json_parse(response.choices[0].message.content)
            violated = random.sample(
                list(attributes.keys()), k=min(random.randint(1, 2), len(attributes))
            ) if attributes else []
            entry["positive_doc"] = parsed.get("positive_doc", "")
            entry["hard_negative_doc"] = parsed.get("hard_negative_doc", "")
            entry["violated_attributes"] = violated
            return entry
        except Exception as e:
            print(f"❌ Error on query_id={entry.get('query_id')}: {e}")
            return None

    tasks = [process_doc(e) for e in data]
    async def limited(entry_task):
        async with semaphore:
            res = await entry_task
            await asyncio.sleep(SLEEP_BETWEEN_REQUESTS)
            return res

    wrapped_tasks = [limited(t) for t in tasks]
    for i, coro in enumerate(asyncio.as_completed(wrapped_tasks), 1):
        res = await coro
        if res:
            processed.append(res)
        if i % SAVE_INTERVAL == 0:
            with open(output_file, "a") as f:
                for p in processed[-SAVE_INTERVAL:]:
                    f.write(json.dumps(p, ensure_ascii=False) + "\n")
            print(f"💾 Saved {i} rewritten docs...")

    with open(output_file, "a") as f:
        for p in processed:
            f.write(json.dumps(p, ensure_ascii=False) + "\n")

    print(f"✅ Finished rewriting {len(processed)} documents!")

# =========================================================
# STEP 7: Run Sequentially (Colab-safe)
# =========================================================
await generate_instructed_queries()
await rewrite_documents()


📘 Loaded 715 core queries
💾 Auto-saved 50 combinations...
💾 Auto-saved 100 combinations...
💾 Auto-saved 150 combinations...
💾 Auto-saved 200 combinations...
💾 Auto-saved 250 combinations...
💾 Auto-saved 300 combinations...
💾 Auto-saved 350 combinations...
💾 Auto-saved 400 combinations...
💾 Auto-saved 450 combinations...
💾 Auto-saved 500 combinations...
💾 Auto-saved 550 combinations...
💾 Auto-saved 600 combinations...
💾 Auto-saved 650 combinations...
💾 Auto-saved 700 combinations...
💾 Auto-saved 750 combinations...
💾 Auto-saved 800 combinations...
💾 Auto-saved 850 combinations...
💾 Auto-saved 900 combinations...
💾 Auto-saved 950 combinations...
💾 Auto-saved 1000 combinations...
💾 Auto-saved 1050 combinations...
💾 Auto-saved 1100 combinations...
💾 Auto-saved 1150 combinations...
💾 Auto-saved 1200 combinations...
💾 Auto-saved 1250 combinations...
💾 Auto-saved 1300 combinations...
💾 Auto-saved 1350 combinations...
💾 Auto-saved 1400 combinations...
💾 Auto-saved 1450 combinations...
💾 Auto-s

In [ ]:
import json

input_file = "4_rewritten_docs_hard_negatives_with_attri.jsonl"          # your input file
output_file = "final_sorted.jsonl"  # sorted output file

with open(input_file, "r") as f:
    data = [json.loads(line) for line in f if line.strip()]

def sort_key(entry):
    qid = str(entry.get("query_id", "")).strip()
    # Try to convert to int if purely numeric, else fallback to string
    return (0, int(qid)) if qid.isdigit() else (1, qid.lower())

data.sort(key=sort_key)

with open(output_file, "w") as f:
    for entry in data:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

print(f"✅ Sorted {len(data)} records by query_id.")
print(f"📁 Output saved to: {output_file}")


**Evaluation Process - J**

In [ ]:
!pip install pyserini tqdm numpy pandas -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.5/178.5 MB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 106.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.0 MB/s eta 0:00:00


**Dense Retrieval Models**

*BM25*

In [ ]:
!pip install rank_bm25 tqdm numpy pandas -q


In [ ]:
import json, numpy as np, pandas as pd
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from collections import defaultdict

# =========================================================
# Load datasets safely
# =========================================================
def safe_load_jsonl(path):
    data = []
    with open(path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError:
                print(f"⚠️ Skipping invalid JSON line {i}")
    return data

with open("query-doc.json", "r", encoding="utf-8") as f:
    qdoc = json.load(f)
queries = safe_load_jsonl("final_sorted.jsonl")

# =========================================================
# Build corpus and mapping
# =========================================================
corpus, doc_ids, qid_map = [], [], {}
for entry in qdoc:
    qid = entry.get("query_id")
    qid_map[qid] = []
    for doc in entry.get("documents", []):
        text = doc.get("text", "").strip()
        if not text:
            continue
        corpus.append(text)
        doc_ids.append(f"{qid}_{doc.get('doc_id', 'unk')}")
        qid_map[qid].append(f"{qid}_{doc.get('doc_id', 'unk')}")

# Tokenize corpus
tokenized_corpus = [c.lower().split() for c in corpus]
bm25 = BM25Okapi(tokenized_corpus)

print(f"✅ Loaded {len(queries)} queries, {len(corpus)} documents.")

# =========================================================
# Retrieval helper
# =========================================================
def retrieve_topk(query, k=10):
    if not query:
        return []
    tokens = query.lower().split()
    scores = bm25.get_scores(tokens)
    ranked_idx = np.argsort(scores)[::-1][:k]
    return [(doc_ids[i], float(scores[i])) for i in ranked_idx]

def find_rank(docid, ranking):
    for i, (d, _) in enumerate(ranking):
        if d == docid:
            return i + 1
    return len(ranking) + 1

# =========================================================
# Compute metrics
# =========================================================
results = []

for q in tqdm(queries, desc="Evaluating BM25"):
    try:
        qid = q.get("query_id")
        if qid not in qid_map:
            continue

        pos_doc = f"{qid}_doc_1"
        Rori = retrieve_topk(q.get("query", ""))
        Rins = retrieve_topk(q.get("instructed_query", ""))
        Rrev = retrieve_topk(q.get("reversed_query", ""))

        # ranks
        Rori_rank = find_rank(pos_doc, Rori)
        Rins_rank = find_rank(pos_doc, Rins)
        Rrev_rank = find_rank(pos_doc, Rrev)

        # ---------- mSICR ----------
        Iq = int((Rins_rank < Rori_rank) and (Rrev_rank > Rori_rank))

        # ---------- mWISE ----------
        m = len(q.get("attributes", {}))
        delta_ins = Rori_rank - Rins_rank
        delta_rev = Rrev_rank - Rori_rank
        reward = delta_ins / max(1, m)
        penalty = -delta_rev / max(1, m)
        mwise = reward + penalty

        # ---------- MDCR ----------
        dims = list(q.get("attributes", {}).keys())
        mq = len(dims)
        pos_field = q.get("positive_doc", "")
        if isinstance(pos_field, dict):
            pos_text = pos_field.get("text", "").lower()
        else:
            pos_text = str(pos_field).lower()

        sat = sum(1 for v in q.get("attributes", {}).values() if str(v).lower() in pos_text)
        strict = 1 if sat == mq and mq > 0 else 0
        soft = sat / mq if mq else 0

        results.append({
            "query_id": qid,
            "mSICR": Iq,
            "mWISE": mwise,
            "MDCR_strict": strict,
            "MDCR_soft": soft
        })
    except Exception as e:
        print(f"⚠️ Skipping query {q.get('query_id', '?')} due to error: {e}")
        continue

# =========================================================
# Aggregate metrics
# =========================================================
df = pd.DataFrame(results)
metrics = {
    "mSICR": df["mSICR"].mean(),
    "mWISE": df["mWISE"].mean(),
    "MDCR_strict": df["MDCR_strict"].mean(),
    "MDCR_soft": df["MDCR_soft"].mean()
}

print("\n📊 BM25 Evaluation Metrics:")
print(json.dumps(metrics, indent=2))


✅ Loaded 9596 queries, 9576 documents.


Evaluating BM25: 100%|██████████| 9596/9596 [39:32<00:00,  4.04it/s]


📊 BM25 Evaluation Metrics:
{
  "mSICR": 0.03511879949979158,
  "mWISE": -0.2748193691816035,
  "MDCR_strict": 0.016673614005835766,
  "MDCR_soft": 0.15713144365707934
}


*Bge-Large-v1.5*

#New


In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch, json, numpy as np, pandas as pd
from tqdm import tqdm

# =========================================================
# Model Config — change for each run
# =========================================================
model_name = "BAAI/bge-large-en-v1.5"  # You can replace this with any embedding model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer(model_name, device=device)

# =========================================================
# Load Data
# =========================================================
with open("query-doc.json", "r") as f:
    qdoc = json.load(f)
with open("final_sorted.jsonl", "r") as f:
    queries = [json.loads(line) for line in f]

# =========================================================
# Build Corpus Embeddings
# =========================================================
corpus = []
doc_ids = []
qid_to_docs = {}

for entry in qdoc:
    qid = entry["query_id"]
    qid_to_docs[qid] = {}
    for doc in entry["documents"]:
        text = doc["text"].strip().lower()
        corpus.append(text)
        doc_id = f"{qid}_{doc['doc_id']}"
        doc_ids.append(doc_id)
        qid_to_docs[qid][text] = doc_id  # Map clean text → doc_id

print(f"Encoding {len(corpus)} documents using {model_name}...")
corpus_embeddings = model.encode(
    corpus, convert_to_tensor=True, batch_size=32, show_progress_bar=True
)

# =========================================================
# Helper Functions
# =========================================================
def retrieve_topk_dense(query, k=10):
    """Retrieve top-k docs for a given query using cosine similarity."""
    query_emb = model.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_emb, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k)
    return [(doc_ids[idx], float(cos_scores[idx])) for idx in top_results.indices]


def find_rank(docid, ranking):
    """Return 1-based rank of a document ID within a ranked list."""
    for i, (d, _) in enumerate(ranking):
        if d == docid:
            return i + 1
    return len(ranking) + 1  # not found

# =========================================================
# Evaluation Loop
# =========================================================
results = []

for q in tqdm(queries, desc=f"Evaluating {model_name}"):
    try:
        qid = q["query_id"]
        pos_text = q.get("positive_doc", "").strip().lower()

        # ---------------------------------------------------
        # 1️⃣ Find Positive Document ID
        # ---------------------------------------------------
        pos_doc = None
        if qid in qid_to_docs:
            for text, did in qid_to_docs[qid].items():
                if text == pos_text:
                    pos_doc = did
                    break

        # Fallback — semantic similarity if exact match fails
        if not pos_doc:
            pos_emb = model.encode(pos_text, convert_to_tensor=True)
            cos_scores = util.cos_sim(pos_emb, corpus_embeddings)[0]
            best_idx = torch.argmax(cos_scores)
            pos_doc = doc_ids[int(best_idx)]

        # ---------------------------------------------------
        # 2️⃣ Retrieve Rankings
        # ---------------------------------------------------
        Rori = retrieve_topk_dense(q["query"])
        Rins = retrieve_topk_dense(q["instructed_query"])
        Rrev = retrieve_topk_dense(q["reversed_query"])

        Rori_rank = find_rank(pos_doc, Rori)
        Rins_rank = find_rank(pos_doc, Rins)
        Rrev_rank = find_rank(pos_doc, Rrev)

        # ---------------------------------------------------
        # 3️⃣ mSICR Calculation
        # ---------------------------------------------------
        Iq = int((Rins_rank < Rori_rank) and (Rrev_rank > Rori_rank))

        # ---------------------------------------------------
        # 4️⃣ mWISE Calculation
        # ---------------------------------------------------
        m = len(q.get("attributes", {})) or 1
        delta_ins = Rori_rank - Rins_rank
        delta_rev = Rrev_rank - Rori_rank
        reward = delta_ins / m
        penalty = -delta_rev / m
        mwise = reward + penalty

        # ---------------------------------------------------
        # 5️⃣ MDCR (Soft + Strict)
        # ---------------------------------------------------
        attrs = q.get("attributes", {})
        mq = len(attrs)

        if mq > 0:
            pos_doc_emb = model.encode(pos_text, convert_to_tensor=True)
            sim_scores = []

            for attr_name, attr_value in attrs.items():
                # Add context for better embedding meaning
                attr_desc = f"The document should reflect {attr_name} = {attr_value}."
                attr_emb = model.encode(attr_desc, convert_to_tensor=True)
                sim = float(util.cos_sim(attr_emb, pos_doc_emb))
                sim_scores.append(sim)

            # Soft MDCR = mean semantic alignment with all attributes
            mdcr_soft = float(np.mean(sim_scores))

            # Strict MDCR = all attribute similarities above threshold
            threshold = max(0.45, np.mean(sim_scores) - 0.05)
            mdcr_strict = int(all(s >= threshold for s in sim_scores))
        else:
            mdcr_soft = 0.0
            mdcr_strict = 0

        # ---------------------------------------------------
        # 6️⃣ Store Results
        # ---------------------------------------------------
        results.append({
            "query_id": qid,
            "mSICR": Iq,
            "mWISE": mwise,
            "MDCR_strict": mdcr_strict,
            "MDCR_soft": mdcr_soft
        })

    except Exception as e:
        print(f"⚠ Skipped query {qid} due to error: {e}")
        continue

# =========================================================
# Aggregate Metrics
# =========================================================
df = pd.DataFrame(results)
metrics = {
    "model": model_name,
    "mSICR": df["mSICR"].mean(),
    "mWISE": df["mWISE"].mean(),
    "MDCR_strict": df["MDCR_strict"].mean(),
    "MDCR_soft": df["MDCR_soft"].mean()
}

print(f"\n📊 {model_name} Evaluation Metrics:")
print(json.dumps(metrics, indent=2))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

⚠ Skipped query 1042639 due to error: 'list' object has no attribute 'strip'


Evaluating BAAI/bge-large-en-v1.5:  87%|████████▋ | 8359/9596 [18:41<01:56, 10.62it/s]

⚠ Skipped query 1042639 due to error: 'list' object has no attribute 'strip'


Evaluating BAAI/bge-large-en-v1.5: 100%|██████████| 9596/9596 [21:40<00:00,  7.38it/s]


📊 BAAI/bge-large-en-v1.5 Evaluation Metrics:
{
  "model": "BAAI/bge-large-en-v1.5",
  "mSICR": 0.06377870563674322,
  "mWISE": -0.25429714683368126,
  "MDCR_strict": 0.18413361169102296,
  "MDCR_soft": 0.4334249242497304
}


*E5-Large-v2*

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch, json, numpy as np, pandas as pd
from tqdm import tqdm

# =========================================================
# Model Config — change for each run
# =========================================================
model_name = "intfloat/e5-large-v2"  # You can replace this with any embedding model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer(model_name, device=device)

# =========================================================
# Load Data
# =========================================================
with open("query-doc.json", "r") as f:
    qdoc = json.load(f)
with open("final_sorted.jsonl", "r") as f:
    queries = [json.loads(line) for line in f]

# =========================================================
# Build Corpus Embeddings
# =========================================================
corpus = []
doc_ids = []
qid_to_docs = {}

for entry in qdoc:
    qid = entry["query_id"]
    qid_to_docs[qid] = {}
    for doc in entry["documents"]:
        text = doc["text"].strip().lower()
        corpus.append(text)
        doc_id = f"{qid}_{doc['doc_id']}"
        doc_ids.append(doc_id)
        qid_to_docs[qid][text] = doc_id  # Map clean text → doc_id

print(f"Encoding {len(corpus)} documents using {model_name}...")
corpus_embeddings = model.encode(
    corpus, convert_to_tensor=True, batch_size=32, show_progress_bar=True
)

# =========================================================
# Helper Functions
# =========================================================
def retrieve_topk_dense(query, k=10):
    """Retrieve top-k docs for a given query using cosine similarity."""
    query_emb = model.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_emb, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k)
    return [(doc_ids[idx], float(cos_scores[idx])) for idx in top_results.indices]


def find_rank(docid, ranking):
    """Return 1-based rank of a document ID within a ranked list."""
    for i, (d, _) in enumerate(ranking):
        if d == docid:
            return i + 1
    return len(ranking) + 1  # not found

# =========================================================
# Evaluation Loop
# =========================================================
results = []

for q in tqdm(queries, desc=f"Evaluating {model_name}"):
    try:
        qid = q["query_id"]
        pos_text = q.get("positive_doc", "").strip().lower()

        # ---------------------------------------------------
        # 1️⃣ Find Positive Document ID
        # ---------------------------------------------------
        pos_doc = None
        if qid in qid_to_docs:
            for text, did in qid_to_docs[qid].items():
                if text == pos_text:
                    pos_doc = did
                    break

        # Fallback — semantic similarity if exact match fails
        if not pos_doc:
            pos_emb = model.encode(pos_text, convert_to_tensor=True)
            cos_scores = util.cos_sim(pos_emb, corpus_embeddings)[0]
            best_idx = torch.argmax(cos_scores)
            pos_doc = doc_ids[int(best_idx)]

        # ---------------------------------------------------
        # 2️⃣ Retrieve Rankings
        # ---------------------------------------------------
        Rori = retrieve_topk_dense(q["query"])
        Rins = retrieve_topk_dense(q["instructed_query"])
        Rrev = retrieve_topk_dense(q["reversed_query"])

        Rori_rank = find_rank(pos_doc, Rori)
        Rins_rank = find_rank(pos_doc, Rins)
        Rrev_rank = find_rank(pos_doc, Rrev)

        # ---------------------------------------------------
        # 3️⃣ mSICR Calculation
        # ---------------------------------------------------
        Iq = int((Rins_rank < Rori_rank) and (Rrev_rank > Rori_rank))

        # ---------------------------------------------------
        # 4️⃣ mWISE Calculation
        # ---------------------------------------------------
        m = len(q.get("attributes", {})) or 1
        delta_ins = Rori_rank - Rins_rank
        delta_rev = Rrev_rank - Rori_rank
        reward = delta_ins / m
        penalty = -delta_rev / m
        mwise = reward + penalty

        # ---------------------------------------------------
        # 5️⃣ MDCR (Soft + Strict)
        # ---------------------------------------------------
        attrs = q.get("attributes", {})
        mq = len(attrs)

        if mq > 0:
            pos_doc_emb = model.encode(pos_text, convert_to_tensor=True)
            sim_scores = []

            for attr_name, attr_value in attrs.items():
                # Add context for better embedding meaning
                attr_desc = f"The document should reflect {attr_name} = {attr_value}."
                attr_emb = model.encode(attr_desc, convert_to_tensor=True)
                sim = float(util.cos_sim(attr_emb, pos_doc_emb))
                sim_scores.append(sim)

            # Soft MDCR = mean semantic alignment with all attributes
            mdcr_soft = float(np.mean(sim_scores))

            # Strict MDCR = all attribute similarities above threshold
            threshold = max(0.45, np.mean(sim_scores) - 0.05)
            mdcr_strict = int(all(s >= threshold for s in sim_scores))
        else:
            mdcr_soft = 0.0
            mdcr_strict = 0

        # ---------------------------------------------------
        # 6️⃣ Store Results
        # ---------------------------------------------------
        results.append({
            "query_id": qid,
            "mSICR": Iq,
            "mWISE": mwise,
            "MDCR_strict": mdcr_strict,
            "MDCR_soft": mdcr_soft
        })

    except Exception as e:
        print(f"⚠ Skipped query {qid} due to error: {e}")
        continue

# =========================================================
# Aggregate Metrics
# =========================================================
df = pd.DataFrame(results)
metrics = {
    "model": model_name,
    "mSICR": df["mSICR"].mean(),
    "mWISE": df["mWISE"].mean(),
    "MDCR_strict": df["MDCR_strict"].mean(),
    "MDCR_soft": df["MDCR_soft"].mean()
}

print(f"\n📊 {model_name} Evaluation Metrics:")
print(json.dumps(metrics, indent=2))


*Instructor-XL*

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch, json, numpy as np, pandas as pd
from tqdm import tqdm

# =========================================================
# Model Config — change for each run
# =========================================================
model_name = "hkunlp/instructor-large"  # You can replace this with any embedding model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer(model_name, device=device)

# =========================================================
# Load Data
# =========================================================
with open("query-doc.json", "r") as f:
    qdoc = json.load(f)
with open("final_sorted.jsonl", "r") as f:
    queries = [json.loads(line) for line in f]

# =========================================================
# Build Corpus Embeddings
# =========================================================
corpus = []
doc_ids = []
qid_to_docs = {}

for entry in qdoc:
    qid = entry["query_id"]
    qid_to_docs[qid] = {}
    for doc in entry["documents"]:
        text = doc["text"].strip().lower()
        corpus.append(text)
        doc_id = f"{qid}_{doc['doc_id']}"
        doc_ids.append(doc_id)
        qid_to_docs[qid][text] = doc_id  # Map clean text → doc_id

print(f"Encoding {len(corpus)} documents using {model_name}...")
corpus_embeddings = model.encode(
    corpus, convert_to_tensor=True, batch_size=32, show_progress_bar=True
)

# =========================================================
# Helper Functions
# =========================================================
def retrieve_topk_dense(query, k=10):
    """Retrieve top-k docs for a given query using cosine similarity."""
    query_emb = model.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_emb, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k)
    return [(doc_ids[idx], float(cos_scores[idx])) for idx in top_results.indices]


def find_rank(docid, ranking):
    """Return 1-based rank of a document ID within a ranked list."""
    for i, (d, _) in enumerate(ranking):
        if d == docid:
            return i + 1
    return len(ranking) + 1  # not found

# =========================================================
# Evaluation Loop
# =========================================================
results = []

for q in tqdm(queries, desc=f"Evaluating {model_name}"):
    try:
        qid = q["query_id"]
        pos_text = q.get("positive_doc", "").strip().lower()

        # ---------------------------------------------------
        # 1️⃣ Find Positive Document ID
        # ---------------------------------------------------
        pos_doc = None
        if qid in qid_to_docs:
            for text, did in qid_to_docs[qid].items():
                if text == pos_text:
                    pos_doc = did
                    break

        # Fallback — semantic similarity if exact match fails
        if not pos_doc:
            pos_emb = model.encode(pos_text, convert_to_tensor=True)
            cos_scores = util.cos_sim(pos_emb, corpus_embeddings)[0]
            best_idx = torch.argmax(cos_scores)
            pos_doc = doc_ids[int(best_idx)]

        # ---------------------------------------------------
        # 2️⃣ Retrieve Rankings
        # ---------------------------------------------------
        Rori = retrieve_topk_dense(q["query"])
        Rins = retrieve_topk_dense(q["instructed_query"])
        Rrev = retrieve_topk_dense(q["reversed_query"])

        Rori_rank = find_rank(pos_doc, Rori)
        Rins_rank = find_rank(pos_doc, Rins)
        Rrev_rank = find_rank(pos_doc, Rrev)

        # ---------------------------------------------------
        # 3️⃣ mSICR Calculation
        # ---------------------------------------------------
        Iq = int((Rins_rank < Rori_rank) and (Rrev_rank > Rori_rank))

        # ---------------------------------------------------
        # 4️⃣ mWISE Calculation
        # ---------------------------------------------------
        m = len(q.get("attributes", {})) or 1
        delta_ins = Rori_rank - Rins_rank
        delta_rev = Rrev_rank - Rori_rank
        reward = delta_ins / m
        penalty = -delta_rev / m
        mwise = reward + penalty

        # ---------------------------------------------------
        # 5️⃣ MDCR (Soft + Strict)
        # ---------------------------------------------------
        attrs = q.get("attributes", {})
        mq = len(attrs)

        if mq > 0:
            pos_doc_emb = model.encode(pos_text, convert_to_tensor=True)
            sim_scores = []

            for attr_name, attr_value in attrs.items():
                # Add context for better embedding meaning
                attr_desc = f"The document should reflect {attr_name} = {attr_value}."
                attr_emb = model.encode(attr_desc, convert_to_tensor=True)
                sim = float(util.cos_sim(attr_emb, pos_doc_emb))
                sim_scores.append(sim)

            # Soft MDCR = mean semantic alignment with all attributes
            mdcr_soft = float(np.mean(sim_scores))

            # Strict MDCR = all attribute similarities above threshold
            threshold = max(0.45, np.mean(sim_scores) - 0.05)
            mdcr_strict = int(all(s >= threshold for s in sim_scores))
        else:
            mdcr_soft = 0.0
            mdcr_strict = 0

        # ---------------------------------------------------
        # 6️⃣ Store Results
        # ---------------------------------------------------
        results.append({
            "query_id": qid,
            "mSICR": Iq,
            "mWISE": mwise,
            "MDCR_strict": mdcr_strict,
            "MDCR_soft": mdcr_soft
        })

    except Exception as e:
        print(f"⚠ Skipped query {qid} due to error: {e}")
        continue

# =========================================================
# Aggregate Metrics
# =========================================================
df = pd.DataFrame(results)
metrics = {
    "model": model_name,
    "mSICR": df["mSICR"].mean(),
    "mWISE": df["mWISE"].mean(),
    "MDCR_strict": df["MDCR_strict"].mean(),
    "MDCR_soft": df["MDCR_soft"].mean()
}

print(f"\n📊 {model_name} Evaluation Metrics:")
print(json.dumps(metrics, indent=2))


*GTE-Qwen2*

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch, json, numpy as np, pandas as pd
from tqdm import tqdm

# =========================================================
# Model Config — change for each run
# =========================================================
model_name = "thenlper/gte-small"  # You can replace this with any embedding model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer(model_name, device=device)

# =========================================================
# Load Data
# =========================================================
with open("query-doc.json", "r") as f:
    qdoc = json.load(f)
with open("final_sorted.jsonl", "r") as f:
    queries = [json.loads(line) for line in f]

# =========================================================
# Build Corpus Embeddings
# =========================================================
corpus = []
doc_ids = []
qid_to_docs = {}

for entry in qdoc:
    qid = entry["query_id"]
    qid_to_docs[qid] = {}
    for doc in entry["documents"]:
        text = doc["text"].strip().lower()
        corpus.append(text)
        doc_id = f"{qid}_{doc['doc_id']}"
        doc_ids.append(doc_id)
        qid_to_docs[qid][text] = doc_id  # Map clean text → doc_id

print(f"Encoding {len(corpus)} documents using {model_name}...")
corpus_embeddings = model.encode(
    corpus, convert_to_tensor=True, batch_size=32, show_progress_bar=True
)

# =========================================================
# Helper Functions
# =========================================================
def retrieve_topk_dense(query, k=10):
    """Retrieve top-k docs for a given query using cosine similarity."""
    query_emb = model.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_emb, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k)
    return [(doc_ids[idx], float(cos_scores[idx])) for idx in top_results.indices]


def find_rank(docid, ranking):
    """Return 1-based rank of a document ID within a ranked list."""
    for i, (d, _) in enumerate(ranking):
        if d == docid:
            return i + 1
    return len(ranking) + 1  # not found

# =========================================================
# Evaluation Loop
# =========================================================
results = []

for q in tqdm(queries, desc=f"Evaluating {model_name}"):
    try:
        qid = q["query_id"]
        pos_text = q.get("positive_doc", "").strip().lower()

        # ---------------------------------------------------
        # 1️⃣ Find Positive Document ID
        # ---------------------------------------------------
        pos_doc = None
        if qid in qid_to_docs:
            for text, did in qid_to_docs[qid].items():
                if text == pos_text:
                    pos_doc = did
                    break

        # Fallback — semantic similarity if exact match fails
        if not pos_doc:
            pos_emb = model.encode(pos_text, convert_to_tensor=True)
            cos_scores = util.cos_sim(pos_emb, corpus_embeddings)[0]
            best_idx = torch.argmax(cos_scores)
            pos_doc = doc_ids[int(best_idx)]

        # ---------------------------------------------------
        # 2️⃣ Retrieve Rankings
        # ---------------------------------------------------
        Rori = retrieve_topk_dense(q["query"])
        Rins = retrieve_topk_dense(q["instructed_query"])
        Rrev = retrieve_topk_dense(q["reversed_query"])

        Rori_rank = find_rank(pos_doc, Rori)
        Rins_rank = find_rank(pos_doc, Rins)
        Rrev_rank = find_rank(pos_doc, Rrev)

        # ---------------------------------------------------
        # 3️⃣ mSICR Calculation
        # ---------------------------------------------------
        Iq = int((Rins_rank < Rori_rank) and (Rrev_rank > Rori_rank))

        # ---------------------------------------------------
        # 4️⃣ mWISE Calculation
        # ---------------------------------------------------
        m = len(q.get("attributes", {})) or 1
        delta_ins = Rori_rank - Rins_rank
        delta_rev = Rrev_rank - Rori_rank
        reward = delta_ins / m
        penalty = -delta_rev / m
        mwise = reward + penalty

        # ---------------------------------------------------
        # 5️⃣ MDCR (Soft + Strict)
        # ---------------------------------------------------
        attrs = q.get("attributes", {})
        mq = len(attrs)

        if mq > 0:
            pos_doc_emb = model.encode(pos_text, convert_to_tensor=True)
            sim_scores = []

            for attr_name, attr_value in attrs.items():
                # Add context for better embedding meaning
                attr_desc = f"The document should reflect {attr_name} = {attr_value}."
                attr_emb = model.encode(attr_desc, convert_to_tensor=True)
                sim = float(util.cos_sim(attr_emb, pos_doc_emb))
                sim_scores.append(sim)

            # Soft MDCR = mean semantic alignment with all attributes
            mdcr_soft = float(np.mean(sim_scores))

            # Strict MDCR = all attribute similarities above threshold
            threshold = max(0.45, np.mean(sim_scores) - 0.05)
            mdcr_strict = int(all(s >= threshold for s in sim_scores))
        else:
            mdcr_soft = 0.0
            mdcr_strict = 0

        # ---------------------------------------------------
        # 6️⃣ Store Results
        # ---------------------------------------------------
        results.append({
            "query_id": qid,
            "mSICR": Iq,
            "mWISE": mwise,
            "MDCR_strict": mdcr_strict,
            "MDCR_soft": mdcr_soft
        })

    except Exception as e:
        print(f"⚠ Skipped query {qid} due to error: {e}")
        continue

# =========================================================
# Aggregate Metrics
# =========================================================
df = pd.DataFrame(results)
metrics = {
    "model": model_name,
    "mSICR": df["mSICR"].mean(),
    "mWISE": df["mWISE"].mean(),
    "MDCR_strict": df["MDCR_strict"].mean(),
    "MDCR_soft": df["MDCR_soft"].mean()
}

print(f"\n📊 {model_name} Evaluation Metrics:")
print(json.dumps(metrics, indent=2))


*E5-Mistral-ins*

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch, json, numpy as np, pandas as pd
from tqdm import tqdm

# =========================================================
# Model Config — change for each run
# =========================================================
model_name = "intfloat/e5-mistral-7b-instruct"  # You can replace this with any embedding model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer(model_name, device=device)

# =========================================================
# Load Data
# =========================================================
with open("query-doc.json", "r") as f:
    qdoc = json.load(f)
with open("final_sorted.jsonl", "r") as f:
    queries = [json.loads(line) for line in f]

# =========================================================
# Build Corpus Embeddings
# =========================================================
corpus = []
doc_ids = []
qid_to_docs = {}

for entry in qdoc:
    qid = entry["query_id"]
    qid_to_docs[qid] = {}
    for doc in entry["documents"]:
        text = doc["text"].strip().lower()
        corpus.append(text)
        doc_id = f"{qid}_{doc['doc_id']}"
        doc_ids.append(doc_id)
        qid_to_docs[qid][text] = doc_id  # Map clean text → doc_id

print(f"Encoding {len(corpus)} documents using {model_name}...")
corpus_embeddings = model.encode(
    corpus, convert_to_tensor=True, batch_size=32, show_progress_bar=True
)

# =========================================================
# Helper Functions
# =========================================================
def retrieve_topk_dense(query, k=10):
    """Retrieve top-k docs for a given query using cosine similarity."""
    query_emb = model.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_emb, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k)
    return [(doc_ids[idx], float(cos_scores[idx])) for idx in top_results.indices]


def find_rank(docid, ranking):
    """Return 1-based rank of a document ID within a ranked list."""
    for i, (d, _) in enumerate(ranking):
        if d == docid:
            return i + 1
    return len(ranking) + 1  # not found

# =========================================================
# Evaluation Loop
# =========================================================
results = []

for q in tqdm(queries, desc=f"Evaluating {model_name}"):
    try:
        qid = q["query_id"]
        pos_text = q.get("positive_doc", "").strip().lower()

        # ---------------------------------------------------
        # 1️⃣ Find Positive Document ID
        # ---------------------------------------------------
        pos_doc = None
        if qid in qid_to_docs:
            for text, did in qid_to_docs[qid].items():
                if text == pos_text:
                    pos_doc = did
                    break

        # Fallback — semantic similarity if exact match fails
        if not pos_doc:
            pos_emb = model.encode(pos_text, convert_to_tensor=True)
            cos_scores = util.cos_sim(pos_emb, corpus_embeddings)[0]
            best_idx = torch.argmax(cos_scores)
            pos_doc = doc_ids[int(best_idx)]

        # ---------------------------------------------------
        # 2️⃣ Retrieve Rankings
        # ---------------------------------------------------
        Rori = retrieve_topk_dense(q["query"])
        Rins = retrieve_topk_dense(q["instructed_query"])
        Rrev = retrieve_topk_dense(q["reversed_query"])

        Rori_rank = find_rank(pos_doc, Rori)
        Rins_rank = find_rank(pos_doc, Rins)
        Rrev_rank = find_rank(pos_doc, Rrev)

        # ---------------------------------------------------
        # 3️⃣ mSICR Calculation
        # ---------------------------------------------------
        Iq = int((Rins_rank < Rori_rank) and (Rrev_rank > Rori_rank))

        # ---------------------------------------------------
        # 4️⃣ mWISE Calculation
        # ---------------------------------------------------
        m = len(q.get("attributes", {})) or 1
        delta_ins = Rori_rank - Rins_rank
        delta_rev = Rrev_rank - Rori_rank
        reward = delta_ins / m
        penalty = -delta_rev / m
        mwise = reward + penalty

        # ---------------------------------------------------
        # 5️⃣ MDCR (Soft + Strict)
        # ---------------------------------------------------
        attrs = q.get("attributes", {})
        mq = len(attrs)

        if mq > 0:
            pos_doc_emb = model.encode(pos_text, convert_to_tensor=True)
            sim_scores = []

            for attr_name, attr_value in attrs.items():
                # Add context for better embedding meaning
                attr_desc = f"The document should reflect {attr_name} = {attr_value}."
                attr_emb = model.encode(attr_desc, convert_to_tensor=True)
                sim = float(util.cos_sim(attr_emb, pos_doc_emb))
                sim_scores.append(sim)

            # Soft MDCR = mean semantic alignment with all attributes
            mdcr_soft = float(np.mean(sim_scores))

            # Strict MDCR = all attribute similarities above threshold
            threshold = max(0.45, np.mean(sim_scores) - 0.05)
            mdcr_strict = int(all(s >= threshold for s in sim_scores))
        else:
            mdcr_soft = 0.0
            mdcr_strict = 0

        # ---------------------------------------------------
        # 6️⃣ Store Results
        # ---------------------------------------------------
        results.append({
            "query_id": qid,
            "mSICR": Iq,
            "mWISE": mwise,
            "MDCR_strict": mdcr_strict,
            "MDCR_soft": mdcr_soft
        })

    except Exception as e:
        print(f"⚠ Skipped query {qid} due to error: {e}")
        continue

# =========================================================
# Aggregate Metrics
# =========================================================
df = pd.DataFrame(results)
metrics = {
    "model": model_name,
    "mSICR": df["mSICR"].mean(),
    "mWISE": df["mWISE"].mean(),
    "MDCR_strict": df["MDCR_strict"].mean(),
    "MDCR_soft": df["MDCR_soft"].mean()
}

print(f"\n📊 {model_name} Evaluation Metrics:")
print(json.dumps(metrics, indent=2))


*GritLM*

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch, json, numpy as np, pandas as pd
from tqdm import tqdm

# =========================================================
# Model Config — change for each run
# =========================================================
model_name = "GritLM/GritLM-7B"  # You can replace this with any embedding model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer(model_name, device=device)

# =========================================================
# Load Data
# =========================================================
with open("query-doc.json", "r") as f:
    qdoc = json.load(f)
with open("final_sorted.jsonl", "r") as f:
    queries = [json.loads(line) for line in f]

# =========================================================
# Build Corpus Embeddings
# =========================================================
corpus = []
doc_ids = []
qid_to_docs = {}

for entry in qdoc:
    qid = entry["query_id"]
    qid_to_docs[qid] = {}
    for doc in entry["documents"]:
        text = doc["text"].strip().lower()
        corpus.append(text)
        doc_id = f"{qid}_{doc['doc_id']}"
        doc_ids.append(doc_id)
        qid_to_docs[qid][text] = doc_id  # Map clean text → doc_id

print(f"Encoding {len(corpus)} documents using {model_name}...")
corpus_embeddings = model.encode(
    corpus, convert_to_tensor=True, batch_size=32, show_progress_bar=True
)

# =========================================================
# Helper Functions
# =========================================================
def retrieve_topk_dense(query, k=10):
    """Retrieve top-k docs for a given query using cosine similarity."""
    query_emb = model.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_emb, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k)
    return [(doc_ids[idx], float(cos_scores[idx])) for idx in top_results.indices]


def find_rank(docid, ranking):
    """Return 1-based rank of a document ID within a ranked list."""
    for i, (d, _) in enumerate(ranking):
        if d == docid:
            return i + 1
    return len(ranking) + 1  # not found

# =========================================================
# Evaluation Loop
# =========================================================
results = []

for q in tqdm(queries, desc=f"Evaluating {model_name}"):
    try:
        qid = q["query_id"]
        pos_text = q.get("positive_doc", "").strip().lower()

        # ---------------------------------------------------
        # 1️⃣ Find Positive Document ID
        # ---------------------------------------------------
        pos_doc = None
        if qid in qid_to_docs:
            for text, did in qid_to_docs[qid].items():
                if text == pos_text:
                    pos_doc = did
                    break

        # Fallback — semantic similarity if exact match fails
        if not pos_doc:
            pos_emb = model.encode(pos_text, convert_to_tensor=True)
            cos_scores = util.cos_sim(pos_emb, corpus_embeddings)[0]
            best_idx = torch.argmax(cos_scores)
            pos_doc = doc_ids[int(best_idx)]

        # ---------------------------------------------------
        # 2️⃣ Retrieve Rankings
        # ---------------------------------------------------
        Rori = retrieve_topk_dense(q["query"])
        Rins = retrieve_topk_dense(q["instructed_query"])
        Rrev = retrieve_topk_dense(q["reversed_query"])

        Rori_rank = find_rank(pos_doc, Rori)
        Rins_rank = find_rank(pos_doc, Rins)
        Rrev_rank = find_rank(pos_doc, Rrev)

        # ---------------------------------------------------
        # 3️⃣ mSICR Calculation
        # ---------------------------------------------------
        Iq = int((Rins_rank < Rori_rank) and (Rrev_rank > Rori_rank))

        # ---------------------------------------------------
        # 4️⃣ mWISE Calculation
        # ---------------------------------------------------
        m = len(q.get("attributes", {})) or 1
        delta_ins = Rori_rank - Rins_rank
        delta_rev = Rrev_rank - Rori_rank
        reward = delta_ins / m
        penalty = -delta_rev / m
        mwise = reward + penalty

        # ---------------------------------------------------
        # 5️⃣ MDCR (Soft + Strict)
        # ---------------------------------------------------
        attrs = q.get("attributes", {})
        mq = len(attrs)

        if mq > 0:
            pos_doc_emb = model.encode(pos_text, convert_to_tensor=True)
            sim_scores = []

            for attr_name, attr_value in attrs.items():
                # Add context for better embedding meaning
                attr_desc = f"The document should reflect {attr_name} = {attr_value}."
                attr_emb = model.encode(attr_desc, convert_to_tensor=True)
                sim = float(util.cos_sim(attr_emb, pos_doc_emb))
                sim_scores.append(sim)

            # Soft MDCR = mean semantic alignment with all attributes
            mdcr_soft = float(np.mean(sim_scores))

            # Strict MDCR = all attribute similarities above threshold
            threshold = max(0.45, np.mean(sim_scores) - 0.05)
            mdcr_strict = int(all(s >= threshold for s in sim_scores))
        else:
            mdcr_soft = 0.0
            mdcr_strict = 0

        # ---------------------------------------------------
        # 6️⃣ Store Results
        # ---------------------------------------------------
        results.append({
            "query_id": qid,
            "mSICR": Iq,
            "mWISE": mwise,
            "MDCR_strict": mdcr_strict,
            "MDCR_soft": mdcr_soft
        })

    except Exception as e:
        print(f"⚠ Skipped query {qid} due to error: {e}")
        continue

# =========================================================
# Aggregate Metrics
# =========================================================
df = pd.DataFrame(results)
metrics = {
    "model": model_name,
    "mSICR": df["mSICR"].mean(),
    "mWISE": df["mWISE"].mean(),
    "MDCR_strict": df["MDCR_strict"].mean(),
    "MDCR_soft": df["MDCR_soft"].mean()
}

print(f"\n📊 {model_name} Evaluation Metrics:")
print(json.dumps(metrics, indent=2))


*SFR-Embedding-2-R*

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch, json, numpy as np, pandas as pd
from tqdm import tqdm

# =========================================================
# Model Config — change for each run
# =========================================================
model_name = "Salesforce/SFR-Embedding-2-R"  # You can replace this with any embedding model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer(model_name, device=device)

# =========================================================
# Load Data
# =========================================================
with open("query-doc.json", "r") as f:
    qdoc = json.load(f)
with open("final_sorted.jsonl", "r") as f:
    queries = [json.loads(line) for line in f]

# =========================================================
# Build Corpus Embeddings
# =========================================================
corpus = []
doc_ids = []
qid_to_docs = {}

for entry in qdoc:
    qid = entry["query_id"]
    qid_to_docs[qid] = {}
    for doc in entry["documents"]:
        text = doc["text"].strip().lower()
        corpus.append(text)
        doc_id = f"{qid}_{doc['doc_id']}"
        doc_ids.append(doc_id)
        qid_to_docs[qid][text] = doc_id  # Map clean text → doc_id

print(f"Encoding {len(corpus)} documents using {model_name}...")
corpus_embeddings = model.encode(
    corpus, convert_to_tensor=True, batch_size=32, show_progress_bar=True
)

# =========================================================
# Helper Functions
# =========================================================
def retrieve_topk_dense(query, k=10):
    """Retrieve top-k docs for a given query using cosine similarity."""
    query_emb = model.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_emb, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k)
    return [(doc_ids[idx], float(cos_scores[idx])) for idx in top_results.indices]


def find_rank(docid, ranking):
    """Return 1-based rank of a document ID within a ranked list."""
    for i, (d, _) in enumerate(ranking):
        if d == docid:
            return i + 1
    return len(ranking) + 1  # not found

# =========================================================
# Evaluation Loop
# =========================================================
results = []

for q in tqdm(queries, desc=f"Evaluating {model_name}"):
    try:
        qid = q["query_id"]
        pos_text = q.get("positive_doc", "").strip().lower()

        # ---------------------------------------------------
        # 1️⃣ Find Positive Document ID
        # ---------------------------------------------------
        pos_doc = None
        if qid in qid_to_docs:
            for text, did in qid_to_docs[qid].items():
                if text == pos_text:
                    pos_doc = did
                    break

        # Fallback — semantic similarity if exact match fails
        if not pos_doc:
            pos_emb = model.encode(pos_text, convert_to_tensor=True)
            cos_scores = util.cos_sim(pos_emb, corpus_embeddings)[0]
            best_idx = torch.argmax(cos_scores)
            pos_doc = doc_ids[int(best_idx)]

        # ---------------------------------------------------
        # 2️⃣ Retrieve Rankings
        # ---------------------------------------------------
        Rori = retrieve_topk_dense(q["query"])
        Rins = retrieve_topk_dense(q["instructed_query"])
        Rrev = retrieve_topk_dense(q["reversed_query"])

        Rori_rank = find_rank(pos_doc, Rori)
        Rins_rank = find_rank(pos_doc, Rins)
        Rrev_rank = find_rank(pos_doc, Rrev)

        # ---------------------------------------------------
        # 3️⃣ mSICR Calculation
        # ---------------------------------------------------
        Iq = int((Rins_rank < Rori_rank) and (Rrev_rank > Rori_rank))

        # ---------------------------------------------------
        # 4️⃣ mWISE Calculation
        # ---------------------------------------------------
        m = len(q.get("attributes", {})) or 1
        delta_ins = Rori_rank - Rins_rank
        delta_rev = Rrev_rank - Rori_rank
        reward = delta_ins / m
        penalty = -delta_rev / m
        mwise = reward + penalty

        # ---------------------------------------------------
        # 5️⃣ MDCR (Soft + Strict)
        # ---------------------------------------------------
        attrs = q.get("attributes", {})
        mq = len(attrs)

        if mq > 0:
            pos_doc_emb = model.encode(pos_text, convert_to_tensor=True)
            sim_scores = []

            for attr_name, attr_value in attrs.items():
                # Add context for better embedding meaning
                attr_desc = f"The document should reflect {attr_name} = {attr_value}."
                attr_emb = model.encode(attr_desc, convert_to_tensor=True)
                sim = float(util.cos_sim(attr_emb, pos_doc_emb))
                sim_scores.append(sim)

            # Soft MDCR = mean semantic alignment with all attributes
            mdcr_soft = float(np.mean(sim_scores))

            # Strict MDCR = all attribute similarities above threshold
            threshold = max(0.45, np.mean(sim_scores) - 0.05)
            mdcr_strict = int(all(s >= threshold for s in sim_scores))
        else:
            mdcr_soft = 0.0
            mdcr_strict = 0

        # ---------------------------------------------------
        # 6️⃣ Store Results
        # ---------------------------------------------------
        results.append({
            "query_id": qid,
            "mSICR": Iq,
            "mWISE": mwise,
            "MDCR_strict": mdcr_strict,
            "MDCR_soft": mdcr_soft
        })

    except Exception as e:
        print(f"⚠ Skipped query {qid} due to error: {e}")
        continue

# =========================================================
# Aggregate Metrics
# =========================================================
df = pd.DataFrame(results)
metrics = {
    "model": model_name,
    "mSICR": df["mSICR"].mean(),
    "mWISE": df["mWISE"].mean(),
    "MDCR_strict": df["MDCR_strict"].mean(),
    "MDCR_soft": df["MDCR_soft"].mean()
}

print(f"\n📊 {model_name} Evaluation Metrics:")
print(json.dumps(metrics, indent=2))


*NV-Embed-v2*

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch, json, numpy as np, pandas as pd
from tqdm import tqdm

# =========================================================
# Model Config — change for each run
# =========================================================
model_name = "nvidia/NV-Embed-v2"  # You can replace this with any embedding model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer(model_name, device=device)

# =========================================================
# Load Data
# =========================================================
with open("query-doc.json", "r") as f:
    qdoc = json.load(f)
with open("final_sorted.jsonl", "r") as f:
    queries = [json.loads(line) for line in f]

# =========================================================
# Build Corpus Embeddings
# =========================================================
corpus = []
doc_ids = []
qid_to_docs = {}

for entry in qdoc:
    qid = entry["query_id"]
    qid_to_docs[qid] = {}
    for doc in entry["documents"]:
        text = doc["text"].strip().lower()
        corpus.append(text)
        doc_id = f"{qid}_{doc['doc_id']}"
        doc_ids.append(doc_id)
        qid_to_docs[qid][text] = doc_id  # Map clean text → doc_id

print(f"Encoding {len(corpus)} documents using {model_name}...")
corpus_embeddings = model.encode(
    corpus, convert_to_tensor=True, batch_size=32, show_progress_bar=True
)

# =========================================================
# Helper Functions
# =========================================================
def retrieve_topk_dense(query, k=10):
    """Retrieve top-k docs for a given query using cosine similarity."""
    query_emb = model.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_emb, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k)
    return [(doc_ids[idx], float(cos_scores[idx])) for idx in top_results.indices]


def find_rank(docid, ranking):
    """Return 1-based rank of a document ID within a ranked list."""
    for i, (d, _) in enumerate(ranking):
        if d == docid:
            return i + 1
    return len(ranking) + 1  # not found

# =========================================================
# Evaluation Loop
# =========================================================
results = []

for q in tqdm(queries, desc=f"Evaluating {model_name}"):
    try:
        qid = q["query_id"]
        pos_text = q.get("positive_doc", "").strip().lower()

        # ---------------------------------------------------
        # 1️⃣ Find Positive Document ID
        # ---------------------------------------------------
        pos_doc = None
        if qid in qid_to_docs:
            for text, did in qid_to_docs[qid].items():
                if text == pos_text:
                    pos_doc = did
                    break

        # Fallback — semantic similarity if exact match fails
        if not pos_doc:
            pos_emb = model.encode(pos_text, convert_to_tensor=True)
            cos_scores = util.cos_sim(pos_emb, corpus_embeddings)[0]
            best_idx = torch.argmax(cos_scores)
            pos_doc = doc_ids[int(best_idx)]

        # ---------------------------------------------------
        # 2️⃣ Retrieve Rankings
        # ---------------------------------------------------
        Rori = retrieve_topk_dense(q["query"])
        Rins = retrieve_topk_dense(q["instructed_query"])
        Rrev = retrieve_topk_dense(q["reversed_query"])

        Rori_rank = find_rank(pos_doc, Rori)
        Rins_rank = find_rank(pos_doc, Rins)
        Rrev_rank = find_rank(pos_doc, Rrev)

        # ---------------------------------------------------
        # 3️⃣ mSICR Calculation
        # ---------------------------------------------------
        Iq = int((Rins_rank < Rori_rank) and (Rrev_rank > Rori_rank))

        # ---------------------------------------------------
        # 4️⃣ mWISE Calculation
        # ---------------------------------------------------
        m = len(q.get("attributes", {})) or 1
        delta_ins = Rori_rank - Rins_rank
        delta_rev = Rrev_rank - Rori_rank
        reward = delta_ins / m
        penalty = -delta_rev / m
        mwise = reward + penalty

        # ---------------------------------------------------
        # 5️⃣ MDCR (Soft + Strict)
        # ---------------------------------------------------
        attrs = q.get("attributes", {})
        mq = len(attrs)

        if mq > 0:
            pos_doc_emb = model.encode(pos_text, convert_to_tensor=True)
            sim_scores = []

            for attr_name, attr_value in attrs.items():
                # Add context for better embedding meaning
                attr_desc = f"The document should reflect {attr_name} = {attr_value}."
                attr_emb = model.encode(attr_desc, convert_to_tensor=True)
                sim = float(util.cos_sim(attr_emb, pos_doc_emb))
                sim_scores.append(sim)

            # Soft MDCR = mean semantic alignment with all attributes
            mdcr_soft = float(np.mean(sim_scores))

            # Strict MDCR = all attribute similarities above threshold
            threshold = max(0.45, np.mean(sim_scores) - 0.05)
            mdcr_strict = int(all(s >= threshold for s in sim_scores))
        else:
            mdcr_soft = 0.0
            mdcr_strict = 0

        # ---------------------------------------------------
        # 6️⃣ Store Results
        # ---------------------------------------------------
        results.append({
            "query_id": qid,
            "mSICR": Iq,
            "mWISE": mwise,
            "MDCR_strict": mdcr_strict,
            "MDCR_soft": mdcr_soft
        })

    except Exception as e:
        print(f"⚠ Skipped query {qid} due to error: {e}")
        continue

# =========================================================
# Aggregate Metrics
# =========================================================
df = pd.DataFrame(results)
metrics = {
    "model": model_name,
    "mSICR": df["mSICR"].mean(),
    "mWISE": df["mWISE"].mean(),
    "MDCR_strict": df["MDCR_strict"].mean(),
    "MDCR_soft": df["MDCR_soft"].mean()
}

print(f"\n📊 {model_name} Evaluation Metrics:")
print(json.dumps(metrics, indent=2))


**Point-wise Re-ranking Models**

*Mistral-ins-v0.2*

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch, json, numpy as np, pandas as pd
from tqdm import tqdm

# =========================================================
# Model Config — change for each run
# =========================================================
model_name = "mistralai/Mistral-7B-Instruct-v0.2"  # You can replace this with any embedding model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer(model_name, device=device)

# =========================================================
# Load Data
# =========================================================
with open("query-doc.json", "r") as f:
    qdoc = json.load(f)
with open("final_sorted.jsonl", "r") as f:
    queries = [json.loads(line) for line in f]

# =========================================================
# Build Corpus Embeddings
# =========================================================
corpus = []
doc_ids = []
qid_to_docs = {}

for entry in qdoc:
    qid = entry["query_id"]
    qid_to_docs[qid] = {}
    for doc in entry["documents"]:
        text = doc["text"].strip().lower()
        corpus.append(text)
        doc_id = f"{qid}_{doc['doc_id']}"
        doc_ids.append(doc_id)
        qid_to_docs[qid][text] = doc_id  # Map clean text → doc_id

print(f"Encoding {len(corpus)} documents using {model_name}...")
corpus_embeddings = model.encode(
    corpus, convert_to_tensor=True, batch_size=32, show_progress_bar=True
)

# =========================================================
# Helper Functions
# =========================================================
def retrieve_topk_dense(query, k=10):
    """Retrieve top-k docs for a given query using cosine similarity."""
    query_emb = model.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_emb, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k)
    return [(doc_ids[idx], float(cos_scores[idx])) for idx in top_results.indices]


def find_rank(docid, ranking):
    """Return 1-based rank of a document ID within a ranked list."""
    for i, (d, _) in enumerate(ranking):
        if d == docid:
            return i + 1
    return len(ranking) + 1  # not found

# =========================================================
# Evaluation Loop
# =========================================================
results = []

for q in tqdm(queries, desc=f"Evaluating {model_name}"):
    try:
        qid = q["query_id"]
        pos_text = q.get("positive_doc", "").strip().lower()

        # ---------------------------------------------------
        # 1️⃣ Find Positive Document ID
        # ---------------------------------------------------
        pos_doc = None
        if qid in qid_to_docs:
            for text, did in qid_to_docs[qid].items():
                if text == pos_text:
                    pos_doc = did
                    break

        # Fallback — semantic similarity if exact match fails
        if not pos_doc:
            pos_emb = model.encode(pos_text, convert_to_tensor=True)
            cos_scores = util.cos_sim(pos_emb, corpus_embeddings)[0]
            best_idx = torch.argmax(cos_scores)
            pos_doc = doc_ids[int(best_idx)]

        # ---------------------------------------------------
        # 2️⃣ Retrieve Rankings
        # ---------------------------------------------------
        Rori = retrieve_topk_dense(q["query"])
        Rins = retrieve_topk_dense(q["instructed_query"])
        Rrev = retrieve_topk_dense(q["reversed_query"])

        Rori_rank = find_rank(pos_doc, Rori)
        Rins_rank = find_rank(pos_doc, Rins)
        Rrev_rank = find_rank(pos_doc, Rrev)

        # ---------------------------------------------------
        # 3️⃣ mSICR Calculation
        # ---------------------------------------------------
        Iq = int((Rins_rank < Rori_rank) and (Rrev_rank > Rori_rank))

        # ---------------------------------------------------
        # 4️⃣ mWISE Calculation
        # ---------------------------------------------------
        m = len(q.get("attributes", {})) or 1
        delta_ins = Rori_rank - Rins_rank
        delta_rev = Rrev_rank - Rori_rank
        reward = delta_ins / m
        penalty = -delta_rev / m
        mwise = reward + penalty

        # ---------------------------------------------------
        # 5️⃣ MDCR (Soft + Strict)
        # ---------------------------------------------------
        attrs = q.get("attributes", {})
        mq = len(attrs)

        if mq > 0:
            pos_doc_emb = model.encode(pos_text, convert_to_tensor=True)
            sim_scores = []

            for attr_name, attr_value in attrs.items():
                # Add context for better embedding meaning
                attr_desc = f"The document should reflect {attr_name} = {attr_value}."
                attr_emb = model.encode(attr_desc, convert_to_tensor=True)
                sim = float(util.cos_sim(attr_emb, pos_doc_emb))
                sim_scores.append(sim)

            # Soft MDCR = mean semantic alignment with all attributes
            mdcr_soft = float(np.mean(sim_scores))

            # Strict MDCR = all attribute similarities above threshold
            threshold = max(0.45, np.mean(sim_scores) - 0.05)
            mdcr_strict = int(all(s >= threshold for s in sim_scores))
        else:
            mdcr_soft = 0.0
            mdcr_strict = 0

        # ---------------------------------------------------
        # 6️⃣ Store Results
        # ---------------------------------------------------
        results.append({
            "query_id": qid,
            "mSICR": Iq,
            "mWISE": mwise,
            "MDCR_strict": mdcr_strict,
            "MDCR_soft": mdcr_soft
        })

    except Exception as e:
        print(f"⚠ Skipped query {qid} due to error: {e}")
        continue

# =========================================================
# Aggregate Metrics
# =========================================================
df = pd.DataFrame(results)
metrics = {
    "model": model_name,
    "mSICR": df["mSICR"].mean(),
    "mWISE": df["mWISE"].mean(),
    "MDCR_strict": df["MDCR_strict"].mean(),
    "MDCR_soft": df["MDCR_soft"].mean()
}

print(f"\n📊 {model_name} Evaluation Metrics:")
print(json.dumps(metrics, indent=2))


*Llama-3.1*

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch, json, numpy as np, pandas as pd
from tqdm import tqdm

# =========================================================
# Model Config — change for each run
# =========================================================
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"  # You can replace this with any embedding model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer(model_name, device=device)

# =========================================================
# Load Data
# =========================================================
with open("query-doc.json", "r") as f:
    qdoc = json.load(f)
with open("final_sorted.jsonl", "r") as f:
    queries = [json.loads(line) for line in f]

# =========================================================
# Build Corpus Embeddings
# =========================================================
corpus = []
doc_ids = []
qid_to_docs = {}

for entry in qdoc:
    qid = entry["query_id"]
    qid_to_docs[qid] = {}
    for doc in entry["documents"]:
        text = doc["text"].strip().lower()
        corpus.append(text)
        doc_id = f"{qid}_{doc['doc_id']}"
        doc_ids.append(doc_id)
        qid_to_docs[qid][text] = doc_id  # Map clean text → doc_id

print(f"Encoding {len(corpus)} documents using {model_name}...")
corpus_embeddings = model.encode(
    corpus, convert_to_tensor=True, batch_size=32, show_progress_bar=True
)

# =========================================================
# Helper Functions
# =========================================================
def retrieve_topk_dense(query, k=10):
    """Retrieve top-k docs for a given query using cosine similarity."""
    query_emb = model.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_emb, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k)
    return [(doc_ids[idx], float(cos_scores[idx])) for idx in top_results.indices]


def find_rank(docid, ranking):
    """Return 1-based rank of a document ID within a ranked list."""
    for i, (d, _) in enumerate(ranking):
        if d == docid:
            return i + 1
    return len(ranking) + 1  # not found

# =========================================================
# Evaluation Loop
# =========================================================
results = []

for q in tqdm(queries, desc=f"Evaluating {model_name}"):
    try:
        qid = q["query_id"]
        pos_text = q.get("positive_doc", "").strip().lower()

        # ---------------------------------------------------
        # 1️⃣ Find Positive Document ID
        # ---------------------------------------------------
        pos_doc = None
        if qid in qid_to_docs:
            for text, did in qid_to_docs[qid].items():
                if text == pos_text:
                    pos_doc = did
                    break

        # Fallback — semantic similarity if exact match fails
        if not pos_doc:
            pos_emb = model.encode(pos_text, convert_to_tensor=True)
            cos_scores = util.cos_sim(pos_emb, corpus_embeddings)[0]
            best_idx = torch.argmax(cos_scores)
            pos_doc = doc_ids[int(best_idx)]

        # ---------------------------------------------------
        # 2️⃣ Retrieve Rankings
        # ---------------------------------------------------
        Rori = retrieve_topk_dense(q["query"])
        Rins = retrieve_topk_dense(q["instructed_query"])
        Rrev = retrieve_topk_dense(q["reversed_query"])

        Rori_rank = find_rank(pos_doc, Rori)
        Rins_rank = find_rank(pos_doc, Rins)
        Rrev_rank = find_rank(pos_doc, Rrev)

        # ---------------------------------------------------
        # 3️⃣ mSICR Calculation
        # ---------------------------------------------------
        Iq = int((Rins_rank < Rori_rank) and (Rrev_rank > Rori_rank))

        # ---------------------------------------------------
        # 4️⃣ mWISE Calculation
        # ---------------------------------------------------
        m = len(q.get("attributes", {})) or 1
        delta_ins = Rori_rank - Rins_rank
        delta_rev = Rrev_rank - Rori_rank
        reward = delta_ins / m
        penalty = -delta_rev / m
        mwise = reward + penalty

        # ---------------------------------------------------
        # 5️⃣ MDCR (Soft + Strict)
        # ---------------------------------------------------
        attrs = q.get("attributes", {})
        mq = len(attrs)

        if mq > 0:
            pos_doc_emb = model.encode(pos_text, convert_to_tensor=True)
            sim_scores = []

            for attr_name, attr_value in attrs.items():
                # Add context for better embedding meaning
                attr_desc = f"The document should reflect {attr_name} = {attr_value}."
                attr_emb = model.encode(attr_desc, convert_to_tensor=True)
                sim = float(util.cos_sim(attr_emb, pos_doc_emb))
                sim_scores.append(sim)

            # Soft MDCR = mean semantic alignment with all attributes
            mdcr_soft = float(np.mean(sim_scores))

            # Strict MDCR = all attribute similarities above threshold
            threshold = max(0.45, np.mean(sim_scores) - 0.05)
            mdcr_strict = int(all(s >= threshold for s in sim_scores))
        else:
            mdcr_soft = 0.0
            mdcr_strict = 0

        # ---------------------------------------------------
        # 6️⃣ Store Results
        # ---------------------------------------------------
        results.append({
            "query_id": qid,
            "mSICR": Iq,
            "mWISE": mwise,
            "MDCR_strict": mdcr_strict,
            "MDCR_soft": mdcr_soft
        })

    except Exception as e:
        print(f"⚠ Skipped query {qid} due to error: {e}")
        continue

# =========================================================
# Aggregate Metrics
# =========================================================
df = pd.DataFrame(results)
metrics = {
    "model": model_name,
    "mSICR": df["mSICR"].mean(),
    "mWISE": df["mWISE"].mean(),
    "MDCR_strict": df["MDCR_strict"].mean(),
    "MDCR_soft": df["MDCR_soft"].mean()
}

print(f"\n📊 {model_name} Evaluation Metrics:")
print(json.dumps(metrics, indent=2))


**List-wise Re-ranking**

*Zephyr-beta*

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch, json, numpy as np, pandas as pd
from tqdm import tqdm

# =========================================================
# Model Config — change for each run
# =========================================================
model_name = "HuggingFaceH4/zephyr-7b-beta"  # You can replace this with any embedding model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer(model_name, device=device)

# =========================================================
# Load Data
# =========================================================
with open("query-doc.json", "r") as f:
    qdoc = json.load(f)
with open("final_sorted.jsonl", "r") as f:
    queries = [json.loads(line) for line in f]

# =========================================================
# Build Corpus Embeddings
# =========================================================
corpus = []
doc_ids = []
qid_to_docs = {}

for entry in qdoc:
    qid = entry["query_id"]
    qid_to_docs[qid] = {}
    for doc in entry["documents"]:
        text = doc["text"].strip().lower()
        corpus.append(text)
        doc_id = f"{qid}_{doc['doc_id']}"
        doc_ids.append(doc_id)
        qid_to_docs[qid][text] = doc_id  # Map clean text → doc_id

print(f"Encoding {len(corpus)} documents using {model_name}...")
corpus_embeddings = model.encode(
    corpus, convert_to_tensor=True, batch_size=32, show_progress_bar=True
)

# =========================================================
# Helper Functions
# =========================================================
def retrieve_topk_dense(query, k=10):
    """Retrieve top-k docs for a given query using cosine similarity."""
    query_emb = model.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_emb, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k)
    return [(doc_ids[idx], float(cos_scores[idx])) for idx in top_results.indices]


def find_rank(docid, ranking):
    """Return 1-based rank of a document ID within a ranked list."""
    for i, (d, _) in enumerate(ranking):
        if d == docid:
            return i + 1
    return len(ranking) + 1  # not found

# =========================================================
# Evaluation Loop
# =========================================================
results = []

for q in tqdm(queries, desc=f"Evaluating {model_name}"):
    try:
        qid = q["query_id"]
        pos_text = q.get("positive_doc", "").strip().lower()

        # ---------------------------------------------------
        # 1️⃣ Find Positive Document ID
        # ---------------------------------------------------
        pos_doc = None
        if qid in qid_to_docs:
            for text, did in qid_to_docs[qid].items():
                if text == pos_text:
                    pos_doc = did
                    break

        # Fallback — semantic similarity if exact match fails
        if not pos_doc:
            pos_emb = model.encode(pos_text, convert_to_tensor=True)
            cos_scores = util.cos_sim(pos_emb, corpus_embeddings)[0]
            best_idx = torch.argmax(cos_scores)
            pos_doc = doc_ids[int(best_idx)]

        # ---------------------------------------------------
        # 2️⃣ Retrieve Rankings
        # ---------------------------------------------------
        Rori = retrieve_topk_dense(q["query"])
        Rins = retrieve_topk_dense(q["instructed_query"])
        Rrev = retrieve_topk_dense(q["reversed_query"])

        Rori_rank = find_rank(pos_doc, Rori)
        Rins_rank = find_rank(pos_doc, Rins)
        Rrev_rank = find_rank(pos_doc, Rrev)

        # ---------------------------------------------------
        # 3️⃣ mSICR Calculation
        # ---------------------------------------------------
        Iq = int((Rins_rank < Rori_rank) and (Rrev_rank > Rori_rank))

        # ---------------------------------------------------
        # 4️⃣ mWISE Calculation
        # ---------------------------------------------------
        m = len(q.get("attributes", {})) or 1
        delta_ins = Rori_rank - Rins_rank
        delta_rev = Rrev_rank - Rori_rank
        reward = delta_ins / m
        penalty = -delta_rev / m
        mwise = reward + penalty

        # ---------------------------------------------------
        # 5️⃣ MDCR (Soft + Strict)
        # ---------------------------------------------------
        attrs = q.get("attributes", {})
        mq = len(attrs)

        if mq > 0:
            pos_doc_emb = model.encode(pos_text, convert_to_tensor=True)
            sim_scores = []

            for attr_name, attr_value in attrs.items():
                # Add context for better embedding meaning
                attr_desc = f"The document should reflect {attr_name} = {attr_value}."
                attr_emb = model.encode(attr_desc, convert_to_tensor=True)
                sim = float(util.cos_sim(attr_emb, pos_doc_emb))
                sim_scores.append(sim)

            # Soft MDCR = mean semantic alignment with all attributes
            mdcr_soft = float(np.mean(sim_scores))

            # Strict MDCR = all attribute similarities above threshold
            threshold = max(0.45, np.mean(sim_scores) - 0.05)
            mdcr_strict = int(all(s >= threshold for s in sim_scores))
        else:
            mdcr_soft = 0.0
            mdcr_strict = 0

        # ---------------------------------------------------
        # 6️⃣ Store Results
        # ---------------------------------------------------
        results.append({
            "query_id": qid,
            "mSICR": Iq,
            "mWISE": mwise,
            "MDCR_strict": mdcr_strict,
            "MDCR_soft": mdcr_soft
        })

    except Exception as e:
        print(f"⚠ Skipped query {qid} due to error: {e}")
        continue

# =========================================================
# Aggregate Metrics
# =========================================================
df = pd.DataFrame(results)
metrics = {
    "model": model_name,
    "mSICR": df["mSICR"].mean(),
    "mWISE": df["mWISE"].mean(),
    "MDCR_strict": df["MDCR_strict"].mean(),
    "MDCR_soft": df["MDCR_soft"].mean()
}

print(f"\n📊 {model_name} Evaluation Metrics:")
print(json.dumps(metrics, indent=2))


*RankVicuna-v1*

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch, json, numpy as np, pandas as pd
from tqdm import tqdm

# =========================================================
# Model Config — change for each run
# =========================================================
model_name = "microsoft/RankVicuna-v1"  # You can replace this with any embedding model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer(model_name, device=device)

# =========================================================
# Load Data
# =========================================================
with open("query-doc.json", "r") as f:
    qdoc = json.load(f)
with open("final_sorted.jsonl", "r") as f:
    queries = [json.loads(line) for line in f]

# =========================================================
# Build Corpus Embeddings
# =========================================================
corpus = []
doc_ids = []
qid_to_docs = {}

for entry in qdoc:
    qid = entry["query_id"]
    qid_to_docs[qid] = {}
    for doc in entry["documents"]:
        text = doc["text"].strip().lower()
        corpus.append(text)
        doc_id = f"{qid}_{doc['doc_id']}"
        doc_ids.append(doc_id)
        qid_to_docs[qid][text] = doc_id  # Map clean text → doc_id

print(f"Encoding {len(corpus)} documents using {model_name}...")
corpus_embeddings = model.encode(
    corpus, convert_to_tensor=True, batch_size=32, show_progress_bar=True
)

# =========================================================
# Helper Functions
# =========================================================
def retrieve_topk_dense(query, k=10):
    """Retrieve top-k docs for a given query using cosine similarity."""
    query_emb = model.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_emb, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k)
    return [(doc_ids[idx], float(cos_scores[idx])) for idx in top_results.indices]


def find_rank(docid, ranking):
    """Return 1-based rank of a document ID within a ranked list."""
    for i, (d, _) in enumerate(ranking):
        if d == docid:
            return i + 1
    return len(ranking) + 1  # not found

# =========================================================
# Evaluation Loop
# =========================================================
results = []

for q in tqdm(queries, desc=f"Evaluating {model_name}"):
    try:
        qid = q["query_id"]
        pos_text = q.get("positive_doc", "").strip().lower()

        # ---------------------------------------------------
        # 1️⃣ Find Positive Document ID
        # ---------------------------------------------------
        pos_doc = None
        if qid in qid_to_docs:
            for text, did in qid_to_docs[qid].items():
                if text == pos_text:
                    pos_doc = did
                    break

        # Fallback — semantic similarity if exact match fails
        if not pos_doc:
            pos_emb = model.encode(pos_text, convert_to_tensor=True)
            cos_scores = util.cos_sim(pos_emb, corpus_embeddings)[0]
            best_idx = torch.argmax(cos_scores)
            pos_doc = doc_ids[int(best_idx)]

        # ---------------------------------------------------
        # 2️⃣ Retrieve Rankings
        # ---------------------------------------------------
        Rori = retrieve_topk_dense(q["query"])
        Rins = retrieve_topk_dense(q["instructed_query"])
        Rrev = retrieve_topk_dense(q["reversed_query"])

        Rori_rank = find_rank(pos_doc, Rori)
        Rins_rank = find_rank(pos_doc, Rins)
        Rrev_rank = find_rank(pos_doc, Rrev)

        # ---------------------------------------------------
        # 3️⃣ mSICR Calculation
        # ---------------------------------------------------
        Iq = int((Rins_rank < Rori_rank) and (Rrev_rank > Rori_rank))

        # ---------------------------------------------------
        # 4️⃣ mWISE Calculation
        # ---------------------------------------------------
        m = len(q.get("attributes", {})) or 1
        delta_ins = Rori_rank - Rins_rank
        delta_rev = Rrev_rank - Rori_rank
        reward = delta_ins / m
        penalty = -delta_rev / m
        mwise = reward + penalty

        # ---------------------------------------------------
        # 5️⃣ MDCR (Soft + Strict)
        # ---------------------------------------------------
        attrs = q.get("attributes", {})
        mq = len(attrs)

        if mq > 0:
            pos_doc_emb = model.encode(pos_text, convert_to_tensor=True)
            sim_scores = []

            for attr_name, attr_value in attrs.items():
                # Add context for better embedding meaning
                attr_desc = f"The document should reflect {attr_name} = {attr_value}."
                attr_emb = model.encode(attr_desc, convert_to_tensor=True)
                sim = float(util.cos_sim(attr_emb, pos_doc_emb))
                sim_scores.append(sim)

            # Soft MDCR = mean semantic alignment with all attributes
            mdcr_soft = float(np.mean(sim_scores))

            # Strict MDCR = all attribute similarities above threshold
            threshold = max(0.45, np.mean(sim_scores) - 0.05)
            mdcr_strict = int(all(s >= threshold for s in sim_scores))
        else:
            mdcr_soft = 0.0
            mdcr_strict = 0

        # ---------------------------------------------------
        # 6️⃣ Store Results
        # ---------------------------------------------------
        results.append({
            "query_id": qid,
            "mSICR": Iq,
            "mWISE": mwise,
            "MDCR_strict": mdcr_strict,
            "MDCR_soft": mdcr_soft
        })

    except Exception as e:
        print(f"⚠ Skipped query {qid} due to error: {e}")
        continue

# =========================================================
# Aggregate Metrics
# =========================================================
df = pd.DataFrame(results)
metrics = {
    "model": model_name,
    "mSICR": df["mSICR"].mean(),
    "mWISE": df["mWISE"].mean(),
    "MDCR_strict": df["MDCR_strict"].mean(),
    "MDCR_soft": df["MDCR_soft"].mean()
}

print(f"\n📊 {model_name} Evaluation Metrics:")
print(json.dumps(metrics, indent=2))


*RankZephyr-v1*

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch, json, numpy as np, pandas as pd
from tqdm import tqdm

# =========================================================
# Model Config — change for each run
# =========================================================
model_name = "HuggingFaceH4/RankZephyr-7b-v1"  # You can replace this with any embedding model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer(model_name, device=device)

# =========================================================
# Load Data
# =========================================================
with open("query-doc.json", "r") as f:
    qdoc = json.load(f)
with open("final_sorted.jsonl", "r") as f:
    queries = [json.loads(line) for line in f]

# =========================================================
# Build Corpus Embeddings
# =========================================================
corpus = []
doc_ids = []
qid_to_docs = {}

for entry in qdoc:
    qid = entry["query_id"]
    qid_to_docs[qid] = {}
    for doc in entry["documents"]:
        text = doc["text"].strip().lower()
        corpus.append(text)
        doc_id = f"{qid}_{doc['doc_id']}"
        doc_ids.append(doc_id)
        qid_to_docs[qid][text] = doc_id  # Map clean text → doc_id

print(f"Encoding {len(corpus)} documents using {model_name}...")
corpus_embeddings = model.encode(
    corpus, convert_to_tensor=True, batch_size=32, show_progress_bar=True
)

# =========================================================
# Helper Functions
# =========================================================
def retrieve_topk_dense(query, k=10):
    """Retrieve top-k docs for a given query using cosine similarity."""
    query_emb = model.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_emb, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k)
    return [(doc_ids[idx], float(cos_scores[idx])) for idx in top_results.indices]


def find_rank(docid, ranking):
    """Return 1-based rank of a document ID within a ranked list."""
    for i, (d, _) in enumerate(ranking):
        if d == docid:
            return i + 1
    return len(ranking) + 1  # not found

# =========================================================
# Evaluation Loop
# =========================================================
results = []

for q in tqdm(queries, desc=f"Evaluating {model_name}"):
    try:
        qid = q["query_id"]
        pos_text = q.get("positive_doc", "").strip().lower()

        # ---------------------------------------------------
        # 1️⃣ Find Positive Document ID
        # ---------------------------------------------------
        pos_doc = None
        if qid in qid_to_docs:
            for text, did in qid_to_docs[qid].items():
                if text == pos_text:
                    pos_doc = did
                    break

        # Fallback — semantic similarity if exact match fails
        if not pos_doc:
            pos_emb = model.encode(pos_text, convert_to_tensor=True)
            cos_scores = util.cos_sim(pos_emb, corpus_embeddings)[0]
            best_idx = torch.argmax(cos_scores)
            pos_doc = doc_ids[int(best_idx)]

        # ---------------------------------------------------
        # 2️⃣ Retrieve Rankings
        # ---------------------------------------------------
        Rori = retrieve_topk_dense(q["query"])
        Rins = retrieve_topk_dense(q["instructed_query"])
        Rrev = retrieve_topk_dense(q["reversed_query"])

        Rori_rank = find_rank(pos_doc, Rori)
        Rins_rank = find_rank(pos_doc, Rins)
        Rrev_rank = find_rank(pos_doc, Rrev)

        # ---------------------------------------------------
        # 3️⃣ mSICR Calculation
        # ---------------------------------------------------
        Iq = int((Rins_rank < Rori_rank) and (Rrev_rank > Rori_rank))

        # ---------------------------------------------------
        # 4️⃣ mWISE Calculation
        # ---------------------------------------------------
        m = len(q.get("attributes", {})) or 1
        delta_ins = Rori_rank - Rins_rank
        delta_rev = Rrev_rank - Rori_rank
        reward = delta_ins / m
        penalty = -delta_rev / m
        mwise = reward + penalty

        # ---------------------------------------------------
        # 5️⃣ MDCR (Soft + Strict)
        # ---------------------------------------------------
        attrs = q.get("attributes", {})
        mq = len(attrs)

        if mq > 0:
            pos_doc_emb = model.encode(pos_text, convert_to_tensor=True)
            sim_scores = []

            for attr_name, attr_value in attrs.items():
                # Add context for better embedding meaning
                attr_desc = f"The document should reflect {attr_name} = {attr_value}."
                attr_emb = model.encode(attr_desc, convert_to_tensor=True)
                sim = float(util.cos_sim(attr_emb, pos_doc_emb))
                sim_scores.append(sim)

            # Soft MDCR = mean semantic alignment with all attributes
            mdcr_soft = float(np.mean(sim_scores))

            # Strict MDCR = all attribute similarities above threshold
            threshold = max(0.45, np.mean(sim_scores) - 0.05)
            mdcr_strict = int(all(s >= threshold for s in sim_scores))
        else:
            mdcr_soft = 0.0
            mdcr_strict = 0

        # ---------------------------------------------------
        # 6️⃣ Store Results
        # ---------------------------------------------------
        results.append({
            "query_id": qid,
            "mSICR": Iq,
            "mWISE": mwise,
            "MDCR_strict": mdcr_strict,
            "MDCR_soft": mdcr_soft
        })

    except Exception as e:
        print(f"⚠ Skipped query {qid} due to error: {e}")
        continue

# =========================================================
# Aggregate Metrics
# =========================================================
df = pd.DataFrame(results)
metrics = {
    "model": model_name,
    "mSICR": df["mSICR"].mean(),
    "mWISE": df["mWISE"].mean(),
    "MDCR_strict": df["MDCR_strict"].mean(),
    "MDCR_soft": df["MDCR_soft"].mean()
}

print(f"\n📊 {model_name} Evaluation Metrics:")
print(json.dumps(metrics, indent=2))


**GPT-4o-mini**

In [ ]:
# ============================================================
# 🧠 GPT-4o-mini Instruction-Following Evaluation (Reranking)
# — Rewritten to mirror the SentenceTransformer script structure
# ============================================================

# pip install openai tqdm numpy pandas -q
import json, numpy as np, pandas as pd, time
from tqdm import tqdm
from openai import AzureOpenAI

# ================== CONFIG ==================
AZURE_ENDPOINT = "https://areypragir-4130-gpt4omi-resource.cognitiveservices.azure.com/"
AZURE_KEY = "##"  # ← put your key here
DEPLOYMENT_NAME = "gpt-4o-mini"
API_VERSION = "2024-08-01-preview"

client = AzureOpenAI(
    azure_endpoint=AZURE_ENDPOINT,
    api_key=AZURE_KEY,
    api_version=API_VERSION
)

# ================== HELPERS & CACHES ==================
gpt_score_cache = {}   # cache for (query_text, doc_id) -> score
gpt_attr_cache = {}    # cache for (attr_desc, doc_id) -> score

def safe_float_from_text(text):
    """Try to parse a float from model output; fallback 0.0."""
    try:
        t = text.strip().split()[0]
        # allow forms like ".8" or "0.8"
        if t.startswith("."):
            t = "0" + t
        return float(t)
    except:
        try:
            return float(text.strip())
        except:
            return 0.0

def gpt_rerank_score(query_text, doc_text, doc_id=None, sleep_if_rate_limited=True):
    """
    Use GPT to return a relevance score between 0 and 1 for (query, doc_text).
    Uses caching keyed by (query_text, doc_id) if doc_id provided, else (query_text, snippet_hash).
    """
    key = (query_text, doc_id if doc_id is not None else hash(doc_text))
    if key in gpt_score_cache:
        return gpt_score_cache[key]

    prompt = f"""You are an expert evaluator judging document relevance for an instruction-following retriever.
Rate how relevant the document below is to the Query on a numeric scale from 0.0 (not relevant at all) to 1.0 (perfectly relevant).
Return only a single number between 0 and 1 (e.g. 0, 0.25, 0.67, 1).

Query:
{query_text}

Document:
{doc_text}
"""
    try:
        resp = client.chat.completions.create(
            model=DEPLOYMENT_NAME,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=10,
            temperature=0.0
        )
        out = resp.choices[0].message.content.strip()
        score = safe_float_from_text(out)
    except Exception as e:
        # basic retry/backoff for transient errors
        # (keep simple, avoid infinite loops)
        if sleep_if_rate_limited:
            time.sleep(1.0)
        try:
            score = 0.0
        except:
            score = 0.0

    # clamp
    score = max(0.0, min(1.0, score))
    gpt_score_cache[key] = score
    return score

def gpt_attr_score(attr_name, attr_value, doc_text, doc_id=None):
    """
    Ask GPT to rate how well `doc_text` reflects the attribute (attr_name=attr_value).
    Returns float in [0,1]. Caches results.
    """
    key = (attr_name, attr_value, doc_id if doc_id is not None else hash(doc_text))
    if key in gpt_attr_cache:
        return gpt_attr_cache[key]

    prompt = f"""You are an expert annotator. Rate how well the following document satisfies the attribute '{attr_name} = {attr_value}'.
Return a single number between 0.0 (not at all) and 1.0 (perfectly).
Document:
{doc_text}
"""
    try:
        resp = client.chat.completions.create(
            model=DEPLOYMENT_NAME,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=10,
            temperature=0.0
        )
        out = resp.choices[0].message.content.strip()
        score = safe_float_from_text(out)
    except Exception as e:
        score = 0.0

    score = max(0.0, min(1.0, score))
    gpt_attr_cache[key] = score
    return score

# ================== LOAD DATA ==================
def safe_load_jsonl(path):
    data = []
    with open(path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError:
                print(f"⚠️ Skipping malformed line {i} in {path}")
    return data

with open("query-doc.json", "r", encoding="utf-8") as f:
    qdoc = json.load(f)
queries = safe_load_jsonl("final_sorted.jsonl")

# Build global corpus, doc_ids, qid_to_docs mapping (same structure as first script)
corpus = []
doc_ids = []
qid_to_docs = {}   # qid -> {clean_text: doc_id}
docid_to_text = {} # doc_id -> original text

for entry in qdoc:
    qid = entry["query_id"]
    qid_to_docs[qid] = {}
    for doc in entry.get("documents", []):
        text = doc["text"].strip().lower()
        doc_id = f"{qid}_{doc.get('doc_id', len(doc_ids))}"
        corpus.append(text)
        doc_ids.append(doc_id)
        qid_to_docs[qid][text] = doc_id
        docid_to_text[doc_id] = text

print(f"Loaded {len(queries)} queries and {len(corpus)} corpus documents.")

# Optional: limit maximum corpus size to score if cost is a problem
MAX_DOCS_TO_SCORE = None  # set to an integer to limit (e.g., 1000). None => score entire corpus.

# ================== Retriever (GPT-based) — mirrors retrieve_topk_dense ==================
def retrieve_topk_gpt(query_text, k=10):
    """
    Score all (or first MAX_DOCS_TO_SCORE) corpus docs using GPT and return top-k list of (doc_id, score).
    Uses caching to avoid duplicate calls.
    """
    results = []
    num_docs = len(corpus) if MAX_DOCS_TO_SCORE is None else min(len(corpus), MAX_DOCS_TO_SCORE)
    # iterate doc_ids with their texts
    for i in range(num_docs):
        did = doc_ids[i]
        text = corpus[i]
        score = gpt_rerank_score(query_text, text, doc_id=did)
        results.append((did, float(score)))
    # sort descending by score
    results.sort(key=lambda x: x[1], reverse=True)
    return results[:k]

def find_rank(docid, ranking):
    """Return 1-based rank of a document ID within a ranked list (ranking is list of (docid,score))."""
    for i, (d, _) in enumerate(ranking):
        if d == docid:
            return i + 1
    return len(ranking) + 1

# ================== Main Evaluation Loop (mirrors first script's flow) ==================
results = []
TOP_K = 30  # for MDCR semantic checks (mirrors earlier script's topk)
RETRIEVE_K = 10

for q in tqdm(queries, desc=f"Evaluating {DEPLOYMENT_NAME} (GPT reranker)"):
    try:
        qid = q["query_id"]
        query_text = q.get("query", "")
        instructed_query = q.get("instructed_query", "")
        reversed_query = q.get("reversed_query", "")
        pos_text = q.get("positive_doc", "").strip().lower()
        attributes = q.get("attributes", {})

        # ---------------------------------------------------
        # 1️⃣ Find Positive Document ID (exact match then GPT fallback)
        # ---------------------------------------------------
        pos_doc = None
        if qid in qid_to_docs:
            for text, did in qid_to_docs[qid].items():
                if text == pos_text:
                    pos_doc = did
                    break

        # Fallback — use GPT to find the most relevant corpus doc to pos_text (semantic fallback)
        if not pos_doc:
            # score pos_text against corpus and pick highest-scoring doc_id
            pos_scores = []
            num_docs = len(corpus) if MAX_DOCS_TO_SCORE is None else min(len(corpus), MAX_DOCS_TO_SCORE)
            for i in range(num_docs):
                did = doc_ids[i]
                text = corpus[i]
                s = gpt_rerank_score(pos_text, text, doc_id=did)
                pos_scores.append((did, float(s)))
            pos_scores.sort(key=lambda x: x[1], reverse=True)
            if pos_scores:
                pos_doc = pos_scores[0][0]
            else:
                # no docs available for this q, skip
                print(f"⚠️ No docs found for query {qid}; skipping.")
                continue

        # ---------------------------------------------------
        # 2️⃣ Retrieve Rankings (global corpus) — mirror retrieve_topk_dense outputs
        # ---------------------------------------------------
        Rori = retrieve_topk_gpt(query_text, k=RETRIEVE_K)
        Rins = retrieve_topk_gpt(instructed_query, k=RETRIEVE_K)
        Rrev = retrieve_topk_gpt(reversed_query, k=RETRIEVE_K)

        Rori_rank = find_rank(pos_doc, Rori)
        Rins_rank = find_rank(pos_doc, Rins)
        Rrev_rank = find_rank(pos_doc, Rrev)

        # ---------------------------------------------------
        # 3️⃣ mSICR Calculation (same boolean condition)
        # ---------------------------------------------------
        Iq = int((Rins_rank < Rori_rank) and (Rrev_rank > Rori_rank))

        # ---------------------------------------------------
        # 4️⃣ mWISE Calculation (same delta / normalized by number of attributes)
        # ---------------------------------------------------
        m = len(attributes) or 1
        delta_ins = Rori_rank - Rins_rank
        delta_rev = Rrev_rank - Rori_rank
        reward = delta_ins / m
        penalty = -delta_rev / m
        mwise = reward + penalty

        # ---------------------------------------------------
        # 5️⃣ MDCR (Soft + Strict) — ask GPT to score positive doc per attribute (0–1)
        # ---------------------------------------------------
        if attributes and len(attributes) > 0:
            pos_doc_text = docid_to_text.get(pos_doc, "")
            sim_scores = []
            for attr_name, attr_value in attributes.items():
                # ask GPT to rate (0.–1.) how well positive doc satisfies attribute
                score = gpt_attr_score(attr_name, attr_value, pos_doc_text, doc_id=pos_doc)
                sim_scores.append(float(score))

            mdcr_soft = float(np.mean(sim_scores)) if sim_scores else 0.0
            threshold = max(0.45, np.mean(sim_scores) - 0.05) if sim_scores else 0.45
            mdcr_strict = int(all(s >= threshold for s in sim_scores))
        else:
            mdcr_soft = 0.0
            mdcr_strict = 0

        # ---------------------------------------------------
        # 6️⃣ Store Results
        # ---------------------------------------------------
        results.append({
            "query_id": qid,
            "mSICR": Iq,
            "mWISE": mwise,
            "MDCR_strict": mdcr_strict,
            "MDCR_soft": mdcr_soft
        })

    except Exception as e:
        print(f"⚠ Skipped query {q.get('query_id','?')} due to error: {e}")
        continue

# =========================================================
# Aggregate Metrics
# =========================================================
df = pd.DataFrame(results)
metrics = {
    "model": DEPLOYMENT_NAME,
    "mSICR": df["mSICR"].mean() if not df.empty else 0.0,
    "mWISE": df["mWISE"].mean() if not df.empty else 0.0,
    "MDCR_strict": df["MDCR_strict"].mean() if not df.empty else 0.0,
    "MDCR_soft": df["MDCR_soft"].mean() if not df.empty else 0.0
}

print(f"\n📊 {DEPLOYMENT_NAME} Evaluation Metrics (SOP-consistent):")
print(json.dumps(metrics, indent=2))


old code

In [ ]:
# ============================================================
# 🧠 GPT-4o-mini Instruction-Following Evaluation (Reranking)
# ============================================================

!pip install openai tqdm numpy pandas -q

import json, numpy as np, pandas as pd
from tqdm import tqdm
from openai import AzureOpenAI

# ================== CONFIG ==================
AZURE_ENDPOINT = "https://areypragir-4130-gpt4omi-resource.cognitiveservices.azure.com/"
AZURE_KEY = "##"
DEPLOYMENT_NAME = "gpt-4o-mini"
API_VERSION = "2024-08-01-preview"

client = AzureOpenAI(
    azure_endpoint=AZURE_ENDPOINT,
    api_key=AZURE_KEY,
    api_version=API_VERSION
)

# ================== GPT SCORE HELPER ==================
def gpt_rerank_score(query, doc):
    """Ask GPT-4o-mini to rate document relevance to query (0–1)."""
    prompt = f"""
You are an expert evaluator judging document relevance for an instruction-following retriever.
Query: {query}

Document:
{doc}

Score from 0 (not relevant at all) to 1 (perfectly relevant). Return only a number.
"""
    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_NAME,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=10,
            temperature=0.0
        )
        score_text = response.choices[0].message.content.strip()
        return float(score_text) if score_text.replace(".", "", 1).isdigit() else 0.0
    except Exception as e:
        print(f"⚠️ GPT error: {e}")
        return 0.0

# ================== LOADERS ==================
def safe_load_jsonl(path):
    data = []
    with open(path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError:
                print(f"⚠️ Skipping malformed line {i}")
    return data

with open("query-doc.json", "r", encoding="utf-8") as f:
    qdoc = json.load(f)
queries = safe_load_jsonl("final_sorted.jsonl")

print(f"✅ Loaded {len(queries)} queries, {len(qdoc)} documents.")

# ================== METRIC HELPERS (SOP FORMULAE) ==================
def find_rank(docid, ranking):
    for i, (d, _) in enumerate(ranking):
        if d == docid:
            return i + 1
    return len(ranking) + 1

def compute_mdcr_semantic(docs, attributes, client, topk=10):
    dims = list(attributes.keys())
    mq = len(dims)
    if mq == 0:
        return 0, 0

    strict, soft = 0, 0
    for doc_text in [d.lower() for d in docs[:topk]]:
        sat = 0
        for key, value in attributes.items():
            prompt = f"Does the following document satisfy the attribute '{key}: {value}'? Answer yes or no.\n\nDocument:\n{doc_text}"
            try:
                resp = client.chat.completions.create(
                    model=DEPLOYMENT_NAME,
                    messages=[{"role": "user", "content": prompt}],
                    max_tokens=3,
                    temperature=0.0
                )
                ans = resp.choices[0].message.content.strip().lower()
                if "yes" in ans:
                    sat += 1
            except:
                continue
        if sat == mq:
            strict = 1
        soft = max(soft, sat / mq)
    return strict, soft

# ================== EVALUATION ==================
results = []
TOP_K = 30

for q in tqdm(queries[:100], desc="Evaluating GPT-4o-mini (sample 100)"):  # limit for cost
    try:
        qid = q.get("query_id")
        query_text = q.get("query", "")
        instructed = q.get("instructed_query", "")
        reversed_q = q.get("reversed_query", "")
        attributes = q.get("attributes", {})

        docs = next((entry["documents"] for entry in qdoc if entry["query_id"] == qid), [])
        if not docs:
            continue

        # compute scores per instruction form
        ori_scores = [gpt_rerank_score(query_text, d["text"]) for d in docs]
        ins_scores = [gpt_rerank_score(instructed, d["text"]) for d in docs]
        rev_scores = [gpt_rerank_score(reversed_q, d["text"]) for d in docs]

        # sort docs by score (descending)
        ori_ranked = sorted(zip([d["text"] for d in docs], ori_scores), key=lambda x: x[1], reverse=True)
        ins_ranked = sorted(zip([d["text"] for d in docs], ins_scores), key=lambda x: x[1], reverse=True)
        rev_ranked = sorted(zip([d["text"] for d in docs], rev_scores), key=lambda x: x[1], reverse=True)

        # ---------- mSICR ----------
        # check if instruction improved ranking but reversed worsened it
        pos_doc = next((d for d in docs if d.get("type") == "positive"), docs[0])
        pos_idx = docs.index(pos_doc)
        Rori_rank = np.argsort(ori_scores)[::-1].tolist().index(pos_idx) + 1
        Rins_rank = np.argsort(ins_scores)[::-1].tolist().index(pos_idx) + 1
        Rrev_rank = np.argsort(rev_scores)[::-1].tolist().index(pos_idx) + 1
        mSICR = int((Rins_rank < Rori_rank) and (Rrev_rank > Rori_rank))

        # ---------- mWISE ----------
        m = len(attributes)
        delta_ins = Rori_rank - Rins_rank
        delta_rev = Rrev_rank - Rori_rank
        mWISE = (delta_ins - delta_rev) / max(1, m)

        # ---------- MDCR (Strict + Soft) ----------
        mdcr_strict, mdcr_soft = compute_mdcr_semantic(
            [doc for doc, _ in ins_ranked], attributes, client, topk=TOP_K
        )

        results.append({
            "query_id": qid,
            "mSICR": mSICR,
            "mWISE": mWISE,
            "MDCR_strict": mdcr_strict,
            "MDCR_soft": mdcr_soft
        })

    except Exception as e:
        print(f"⚠️ Skipped {q.get('query_id','?')} due to {e}")
        continue

# ================== METRIC AGGREGATION ==================
df = pd.DataFrame(results)
metrics = {
    "model": "gpt-4o-mini",
    "mSICR": df["mSICR"].mean(),
    "mWISE": df["mWISE"].mean(),
    "MDCR_strict": df["MDCR_strict"].mean(),
    "MDCR_soft": df["MDCR_soft"].mean()
}

print("\n📊 GPT-4o-mini Evaluation Metrics (SOP-consistent):")
print(json.dumps(metrics, indent=2))

✅ Loaded 9596 queries, 515 documents.


Evaluating GPT-4o-mini (sample 100): 100%|██████████| 100/100 [2:15:23<00:00, 81.23s/it] 


📊 GPT-4o-mini Evaluation Metrics (SOP-consistent):
{
  "model": "gpt-4o-mini",
  "mSICR": 0.34,
  "mWISE": -0.23833333333333345,
  "MDCR_strict": 0.75,
  "MDCR_soft": 0.905
}


**NEW MODEL**

In [ ]:
import json
from pathlib import Path
from tqdm import tqdm

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer
import os; os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# ----------------------------
# Configuration & hyperparams
# ----------------------------
FINAL_SORTED = "final_sorted.jsonl"
OUTPUT_DIR = "mif_embed_checkpoints"
device = "cuda" if torch.cuda.is_available() else "cpu"

BASE_MODEL = "sentence-transformers/all-MiniLM-L6-v2"   # try BGE/E5 too
PASSAGE_MODEL_NAME = BASE_MODEL
INSTR_MODEL_NAME = BASE_MODEL

batch_size = 8
epochs = 3
lr = 2e-5
max_len = 128
temperature = 0.07
save_every = 1  # epochs

# ----------------------------
# Utilities to build I and IQ
# ----------------------------
def build_instruction_text(attrs: dict) -> str:
    if not attrs:
        return ""
    parts = [f"{k.strip()} = {str(v).strip()}" for k, v in attrs.items()]
    return "; ".join(parts)

def build_iq_text(attrs: dict, instructed_query: str) -> str:
    instr = build_instruction_text(attrs)
    return f"Attributes: {instr} || Query: {instructed_query}" if instr else f"Query: {instructed_query}"

# ----------------------------
# Dataset
# ----------------------------
class MIFDataset(Dataset):
    def __init__(self, final_sorted_path: str):
        self.entries = []
        p = Path(final_sorted_path)
        assert p.exists(), f"{final_sorted_path} not found"
        with p.open("r", encoding="utf-8") as f:
            for line in f:
                if not line.strip():
                    continue
                obj = json.loads(line)
                if not obj.get("instructed_query") or not obj.get("positive_doc"):
                    continue
                attrs = obj.get("attributes", {})
                P = obj["positive_doc"]
                I = build_instruction_text(attrs)
                IQ = build_iq_text(attrs, obj["instructed_query"])
                self.entries.append({"P": P, "I": I, "IQ": IQ})

    def __len__(self):
        return len(self.entries)

    def __getitem__(self, idx):
        e = self.entries[idx]
        return e["P"], e["I"], e["IQ"]

def collate_fn(batch):
    Ps, Is, IQs = zip(*batch)
    return list(Ps), list(Is), list(IQs)

# ----------------------------
# Encoders
# ----------------------------
def make_encoder(model_name, max_seq_length=256):
    model = SentenceTransformer(model_name, device=device)
    model.max_seq_length = max_seq_length
    return model

# ----------------------------
# Loss helpers
# ----------------------------
def cosine_sim(a: torch.Tensor, b: torch.Tensor):
    a = F.normalize(a, p=2, dim=1)
    b = F.normalize(b, p=2, dim=1)
    return a @ b.t()

def info_nce_loss(sim_matrix: torch.Tensor, temperature: float = 0.07):
    logits = sim_matrix / temperature
    labels = torch.arange(sim_matrix.size(0), device=sim_matrix.device)
    return F.cross_entropy(logits, labels)

# ----------------------------
# Encode with gradients
# ----------------------------
def encode_trainable(st_model: SentenceTransformer, texts: list[str]) -> torch.Tensor:
    # 1) tokenize
    batch_features = st_model.tokenize(texts)
    batch_features = {k: v.to(device) for k, v in batch_features.items()}
    # 2) forward
    out = st_model(batch_features)
    # 3) sentence embedding
    return out["sentence_embedding"]

# ----------------------------
# Main training
# ----------------------------
def train():
    dataset = MIFDataset(FINAL_SORTED)
    print("Dataset size:", len(dataset))
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=True,
        collate_fn=collate_fn, drop_last=True, pin_memory=True
    )

    passage_encoder = make_encoder(PASSAGE_MODEL_NAME, max_seq_length=max_len)
    instr_encoder = make_encoder(INSTR_MODEL_NAME, max_seq_length=max_len)

    passage_encoder.train()
    instr_encoder.train()

    params = list(passage_encoder.parameters()) + list(instr_encoder.parameters())
    optimizer = torch.optim.AdamW(params, lr=lr, weight_decay=0.01)

    global_step = 0
    for epoch in range(1, epochs + 1):
        epoch_loss = 0.0
        pbar = tqdm(dataloader, desc=f"Epoch {epoch}/{epochs}")
        for Ps, Is, IQs in pbar:
            # Forward with grads (NO encode(); use forward path)
            P_embs  = encode_trainable(passage_encoder, Ps)
            I_embs  = encode_trainable(instr_encoder, Is)
            IQ_embs = encode_trainable(instr_encoder, IQs)

            # 1) P vs IQ
            sim_P_IQ = cosine_sim(P_embs, IQ_embs)
            L_p = info_nce_loss(sim_P_IQ, temperature)

            # 2) P vs I
            sim_P_I = cosine_sim(P_embs, I_embs)
            L_i = info_nce_loss(sim_P_I, temperature)

            # 3) IQ vs P (symmetric)
            sim_IQ_P = cosine_sim(IQ_embs, P_embs)
            L_iq = info_nce_loss(sim_IQ_P, temperature)

            loss = (L_p + L_i + L_iq) / 3.0

            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            optimizer.step()

            epoch_loss += float(loss.item())
            global_step += 1
            pbar.set_postfix({"loss": f"{epoch_loss / global_step:.5f}"})

        avg_loss = epoch_loss / max(1, global_step)
        print(f"Epoch {epoch} finished. Avg loss: {avg_loss:.6f}")

        # Save
        if epoch % save_every == 0:
            out_dir = Path(OUTPUT_DIR) / f"epoch_{epoch}"
            out_dir.mkdir(parents=True, exist_ok=True)
            passage_encoder.save(str(out_dir / "passage_encoder"))
            instr_encoder.save(str(out_dir / "instr_encoder"))
            print(f"✓ Saved checkpoint to {out_dir}")

    print("✅ Training finished.")

if __name__ == "__main__":
    train()


Dataset size: 9596


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 46.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 4.12 MiB is free. Process 2600 has 14.73 GiB memory in use. Of the allocated memory 14.22 GiB is allocated by PyTorch, and 397.78 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
"""
mif_evaluate.py
Evaluate fine-tuned MIF-style multi-instruction retrieval model
on your multi-attribute dataset (final_sorted.jsonl + query-doc.json)
using mSICR, mWISE, MDCR_strict, MDCR_soft.

Author: ChatGPT for Pragna
"""
from sentence_transformers import SentenceTransformer, util
import torch, json, numpy as np, pandas as pd
from tqdm import tqdm
from pathlib import Path

# =========================================================
# Config
# =========================================================
PASSAGE_ENCODER_PATH = "mif_embed_checkpoints/epoch_3/passage_encoder"
INSTR_ENCODER_PATH = "mif_embed_checkpoints/epoch_3/instr_encoder"

device = "cuda" if torch.cuda.is_available() else "cpu"
passage_encoder = SentenceTransformer(PASSAGE_ENCODER_PATH, device=device)
instr_encoder = SentenceTransformer(INSTR_ENCODER_PATH, device=device)

# =========================================================
# Load Data
# =========================================================
with open("query-doc.json", "r") as f:
    qdoc = json.load(f)
with open("final_sorted.jsonl", "r") as f:
    queries = [json.loads(line) for line in f]

# =========================================================
# Build Corpus Embeddings
# =========================================================
corpus = []
doc_ids = []
qid_to_docs = {}

for entry in qdoc:
    qid = entry["query_id"]
    qid_to_docs[qid] = {}
    for doc in entry["documents"]:
        corpus.append(doc["text"])
        doc_id = f"{qid}_{doc['doc_id']}"
        doc_ids.append(doc_id)
        qid_to_docs[qid][doc["text"].strip().lower()] = doc_id  # map text → id

print(f"Encoding {len(corpus)} documents with passage_encoder...")
corpus_embeddings = passage_encoder.encode(
    corpus, convert_to_tensor=True, batch_size=32, show_progress_bar=True
)

# =========================================================
# Helper functions
# =========================================================
def retrieve_topk_dense(query_text, k=10):
    """Return top-k ranked documents for a given query using cosine similarity."""
    query_emb = instr_encoder.encode(query_text, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_emb, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k)
    return [(doc_ids[idx], float(cos_scores[idx])) for idx in top_results.indices]

def find_rank(docid, ranking):
    for i, (d, _) in enumerate(ranking):
        if d == docid:
            return i + 1
    return len(ranking) + 1

# =========================================================
# Evaluation Loop
# =========================================================
results = []
for q in tqdm(queries, desc="Evaluating MIF Model"):
    try:
        qid = q["query_id"]
        pos_text = q.get("positive_doc", "").strip().lower()

        # Find gold doc ID
        pos_doc = None
        if qid in qid_to_docs:
            for text, did in qid_to_docs[qid].items():
                if text == pos_text:
                    pos_doc = did
                    break
        if not pos_doc:
            continue

        # Retrieve
        Rori = retrieve_topk_dense(q["query"])
        Rins = retrieve_topk_dense(q["instructed_query"])
        Rrev = retrieve_topk_dense(q["reversed_query"])

        Rori_rank = find_rank(pos_doc, Rori)
        Rins_rank = find_rank(pos_doc, Rins)
        Rrev_rank = find_rank(pos_doc, Rrev)

        # mSICR (multi-instruction compliance ratio)
        mSICR = int((Rins_rank < Rori_rank) and (Rrev_rank > Rori_rank))

        # mWISE
        m = len(q.get("attributes", {})) or 1
        delta_ins = Rori_rank - Rins_rank
        delta_rev = Rrev_rank - Rori_rank
        reward = delta_ins / m
        penalty = -delta_rev / m
        mWISE = reward + penalty

        # MDCR (strict & soft)
        attrs = q.get("attributes", {})
        mq = len(attrs)
        if mq > 0:
            pos_doc_emb = passage_encoder.encode(pos_text, convert_to_tensor=True)
            sim_scores = []
            for k, v in attrs.items():
                attr_desc = f"{k}: {v}"
                attr_emb = instr_encoder.encode(attr_desc, convert_to_tensor=True)
                sim_scores.append(float(util.cos_sim(attr_emb, pos_doc_emb)))
            mdcr_soft = np.mean(sim_scores)
            mdcr_strict = int(all(s >= 0.5 for s in sim_scores))
        else:
            mdcr_soft = 0
            mdcr_strict = 0

        results.append({
            "query_id": qid,
            "mSICR": mSICR,
            "mWISE": mWISE,
            "MDCR_strict": mdcr_strict,
            "MDCR_soft": mdcr_soft
        })

    except Exception as e:
        print(f"⚠ Skipped {qid}: {e}")
        continue

# =========================================================
# Aggregate Metrics
# =========================================================
df = pd.DataFrame(results)
metrics = {
    "model": "MIF-MultiInstruction",
    "mSICR": df["mSICR"].mean(),
    "mWISE": df["mWISE"].mean(),
    "MDCR_strict": df["MDCR_strict"].mean(),
    "MDCR_soft": df["MDCR_soft"].mean()
}
print(f"\n📊 Evaluation Results:")
print(json.dumps(metrics, indent=2))

FileNotFoundError: Path mif_embed_checkpoints/epoch_3/passage_encoder not found

In [ ]:
!pip install -q torch torchvision transformers datasets faiss-cpu tqdm scikit-learn


In [ ]:
import json, random, os
from typing import List, Dict
from tqdm import tqdm
import numpy as np
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from torch.optim import AdamW
import faiss
from sklearn.metrics import ndcg_score


In [ ]:
def mrr_at_k(ranks, k):
    return float(np.mean([1.0/r if r<=k else 0.0 for r in ranks]))

def recall_at_k(ranks, k):
    return float(np.mean([1 if r<=k else 0 for r in ranks]))

def compute_metrics_for_queries(ranks):
    return {
        "MRR@10": mrr_at_k(ranks,10),
        "Recall@1": recall_at_k(ranks,1),
        "Recall@5": recall_at_k(ranks,5),
        "Recall@10": recall_at_k(ranks,10)
    }


In [ ]:
DATA_PATH = "query-doc.json"   # change if path differs

def load_query_doc_json(path):
    with open(path,"r",encoding="utf-8") as f:
        data = json.load(f)
    examples=[]
    for item in data:
        q=item["query"]
        pos=[d["text"] for d in item["documents"] if d["type"].lower()=="positive"]
        hneg=[d["text"] for d in item["documents"] if d["type"].lower()=="hard_negative"]
        other=[d["text"] for d in item["documents"] if d["type"].lower() not in ["positive","hard_negative"]]
        if pos:
            examples.append({"query":q,"positives":pos,"hard_negs":hneg,"other_negs":other})
    return examples

examples=load_query_doc_json(DATA_PATH)
print("Total examples:",len(examples))


Total examples: 515


In [ ]:
def train_dev_split(data,dev_frac=0.1,seed=42):
    random.Random(seed).shuffle(data)
    n_dev=max(1,int(len(data)*dev_frac))
    return data[n_dev:], data[:n_dev]

train_exs, dev_exs = train_dev_split(examples)
print(len(train_exs),"train,",len(dev_exs),"dev")


464 train, 51 dev


In [ ]:
class RetrieverDataset(Dataset):
    def __init__(self,exs,max_hard_negs=1):
        self.exs=exs; self.max_hard_negs=max_hard_negs
    def __len__(self): return len(self.exs)
    def __getitem__(self,idx):
        ex=self.exs[idx]
        pos=random.choice(ex["positives"])
        hns=ex["hard_negs"][:self.max_hard_negs] if ex["hard_negs"] else []
        return {"query":ex["query"],"positive":pos,"hard_negs":hns}


In [ ]:
class DualEncoder(nn.Module):
    def __init__(self,backbone="sentence-transformers/all-MiniLM-L6-v2",
                 proj_dim=256,device="cpu"):
        super().__init__()
        self.device=device
        self.backbone=AutoModel.from_pretrained(backbone)
        hid=self.backbone.config.hidden_size
        self.proj=nn.Linear(hid,proj_dim)
        self.tok=AutoTokenizer.from_pretrained(backbone)
    def encode(self,texts,maxlen=256):
        tok=self.tok(texts,padding=True,truncation=True,max_length=maxlen,return_tensors="pt").to(self.device)
        out=self.backbone(**tok,return_dict=True)
        mask=tok["attention_mask"].unsqueeze(-1).float()
        mean=(out.last_hidden_state*mask).sum(1)/mask.sum(1).clamp(min=1e-9)
        emb=F.normalize(self.proj(mean),dim=-1)
        return emb


In [ ]:
def collate(batch):
    queries=[b["query"] for b in batch]
    positives=[b["positive"] for b in batch]
    pos_idx=list(range(len(batch)))
    hardnegs=[hn for b in batch for hn in b["hard_negs"]]
    passages=positives+hardnegs
    return {"queries":queries,"passages":passages,"pos_idx":pos_idx,"B":len(batch)}

def train_epoch(model,loader,opt,sched,device,temp=0.05):
    model.train(); total=0
    for batch in tqdm(loader,desc="train"):
        c=collate(batch)
        q_emb=model.encode(c["queries"])
        p_emb=model.encode(c["passages"])
        sims=(q_emb@p_emb.T)/temp
        labels=torch.arange(c["B"],device=device)
        loss=F.cross_entropy(sims,labels)
        opt.zero_grad(); loss.backward(); opt.step()
        if sched: sched.step()
        total+=loss.item()*c["B"]
    return total/len(loader.dataset)


In [ ]:
def evaluate(model,dev,device):
    model.eval(); ranks=[]
    with torch.no_grad():
        for ex in tqdm(dev,desc="eval"):
            q=ex["query"]
            cands=ex["positives"]+ex["hard_negs"]+ex["other_negs"]
            if not cands: continue
            q_emb=model.encode([q])
            p_emb=model.encode(cands)
            sims=(q_emb@p_emb.T).squeeze(0).cpu().numpy()
            order=np.argsort(-sims)
            posset=set(range(len(ex["positives"])))
            rank=min((i+1 for i,j in enumerate(order) if j in posset),default=len(cands)+1)
            ranks.append(rank)
    return compute_metrics_for_queries(ranks)


In [ ]:
device="cuda" if torch.cuda.is_available() else "cpu"
print("Using",device)

BATCH=8
EPOCHS=2
LR=5e-5

train_ds=RetrieverDataset(train_exs,max_hard_negs=1)
train_loader=DataLoader(train_ds,batch_size=BATCH,shuffle=True,collate_fn=lambda x:x)

model=DualEncoder(device=device).to(device)
opt=AdamW(model.parameters(),lr=LR,weight_decay=0.01)
steps=len(train_loader)*EPOCHS
sched=get_linear_schedule_with_warmup(opt,int(0.1*steps),steps)

best=0
for ep in range(EPOCHS):
    print(f"=== Epoch {ep+1}/{EPOCHS} ===")
    loss=train_epoch(model,train_loader,opt,sched,device)
    print("Loss:",loss)
    metrics=evaluate(model,dev_exs,device)
    print("Dev:",metrics)
    if metrics["MRR@10"]>best:
        best=metrics["MRR@10"]
        torch.save(model.state_dict(),"best_model.pt")
        print("✅ Saved best model")


Using cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

...

Dev: {'MRR@10': 0.9411764705882353, 'Recall@1': 0.9019607843137255, 'Recall@5': 1.0, 'Recall@10': 1.0}
✅ Saved best model
=== Epoch 2/2 ===


train: 100%|██████████| 58/58 [00:07<00:00,  7.99it/s]


Loss: 0.13477179269980769


eval: 100%|██████████| 51/51 [00:01<00:00, 27.60it/s]


Dev: {'MRR@10': 0.957516339869281, 'Recall@1': 0.9215686274509803, 'Recall@5': 1.0, 'Recall@10': 1.0}
✅ Saved best model


In [ ]:
def build_index(model,texts,device):
    model.eval(); allv=[]
    with torch.no_grad():
        for i in range(0,len(texts),128):
            allv.append(model.encode(texts[i:i+128]).cpu().numpy())
    embs=np.vstack(allv).astype("float32")
    dim=embs.shape[1]
    idx=faiss.IndexFlatIP(dim)
    idx.add(embs)
    return idx,embs

# Build a simple corpus from all docs
corpus = list({
    text
    for ex in examples
    for group in [ex["positives"], ex["hard_negs"], ex["other_negs"]]
    for text in group
})

index,_=build_index(model,corpus,device)
print("Corpus size:",len(corpus))

# Test retrieval for one dev query
sample=dev_exs[0]["query"]
q_emb = model.encode([sample]).detach().cpu().numpy()
scores,ids=index.search(q_emb,5)
print("Query:",sample)
for s,i in zip(scores[0],ids[0]):
    print(f"{s:.3f} | {corpus[i][:150]!r}")


Corpus size: 9576
Query: weather in gru
0.685 | 'Weekly Weather Overview for GRU, São Paulo. The weather at GRU, São Paulo, Brazil, shows that the highest temperature will be 33 degrees Celsius on a '
0.636 | 'Weekly Weather Report for Guarulhos International Airport, São Paulo, Brazil.\n\nThis report provides an overview of the weather conditions expected at '
0.632 | 'Weekly Weather Report for GRU, São Paulo, Brazil. Over the next 7 days, the weather at Guarulhos International Airport is expected to vary. The maximu'
0.620 | 'Weekly Weather Report for GRU, São Paulo. The weather forecast for the upcoming week indicates a maximum temperature of 33℃ (91℉) on Wednesday, August'
0.618 | 'Weekly Weather Report for GRU, São Paulo, Brazil. This report provides an overview of the weather at Guarulhos International Airport over the next 7 d'


In [ ]:
# =========================================================
# 1️⃣ Imports
# =========================================================
import torch, torch.nn as nn, torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import util
import json, numpy as np, pandas as pd
from tqdm import tqdm

# =========================================================
# 2️⃣ Define DualEncoder (same as your training model)
# =========================================================
class DualEncoder(nn.Module):
    def __init__(self, backbone="sentence-transformers/all-MiniLM-L6-v2", proj_dim=256, device="cpu"):
        super().__init__()
        self.device = device
        self.backbone = AutoModel.from_pretrained(backbone)
        hid = self.backbone.config.hidden_size
        self.proj = nn.Linear(hid, proj_dim)
        self.tok = AutoTokenizer.from_pretrained(backbone)

    def encode(self, texts, maxlen=256):
        tok = self.tok(texts, padding=True, truncation=True, max_length=maxlen, return_tensors="pt").to(self.device)
        out = self.backbone(**tok, return_dict=True)
        mask = tok["attention_mask"].unsqueeze(-1).float()
        mean = (out.last_hidden_state * mask).sum(1) / mask.sum(1).clamp(min=1e-9)
        emb = F.normalize(self.proj(mean), dim=-1)
        return emb

# =========================================================
# 3️⃣ Load fine-tuned model
# =========================================================
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

backbone = "sentence-transformers/all-MiniLM-L6-v2"
model = DualEncoder(backbone=backbone, device=device).to(device)
model.load_state_dict(torch.load("best_model.pt", map_location=device))
model.eval()

# Helper for batched encoding
def encode(texts, batch_size=32):
    all_embs = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        with torch.no_grad():
            emb = model.encode(batch).detach().cpu()
        all_embs.append(emb)
    return torch.cat(all_embs, dim=0)

# =========================================================
# 4️⃣ Load dataset
# =========================================================
with open("query-doc.json", "r") as f:
    qdoc = json.load(f)
with open("final_sorted.jsonl", "r") as f:
    queries = [json.loads(line) for line in f]

# =========================================================
# 5️⃣ Build corpus embeddings
# =========================================================
corpus, doc_ids, qid_to_docs = [], [], {}
for entry in qdoc:
    qid = entry["query_id"]
    qid_to_docs[qid] = {}
    for doc in entry["documents"]:
        text = doc["text"].strip().lower()
        corpus.append(text)
        doc_id = f"{qid}_{doc['doc_id']}"
        doc_ids.append(doc_id)
        qid_to_docs[qid][text] = doc_id

print(f"Encoding {len(corpus)} documents using fine-tuned model...")
corpus_embeddings = encode(corpus, batch_size=32)

# =========================================================
# 6️⃣ Retrieval helpers
# =========================================================
def retrieve_topk(query, k=10):
    query_emb = encode([query])
    cos_scores = util.cos_sim(query_emb, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k)
    return [(doc_ids[idx], float(cos_scores[idx])) for idx in top_results.indices]

def find_rank(docid, ranking):
    for i, (d, _) in enumerate(ranking):
        if d == docid:
            return i + 1
    return len(ranking) + 1

# =========================================================
# 7️⃣ Evaluation loop (mSICR, mWISE, MDCR)
# =========================================================
results = []

for q in tqdm(queries, desc="Evaluating fine-tuned model"):
    try:
        qid = q["query_id"]
        pos_text = q.get("positive_doc", "").strip().lower()

        # Match positive doc
        pos_doc = None
        if qid in qid_to_docs:
            for text, did in qid_to_docs[qid].items():
                if text == pos_text:
                    pos_doc = did
                    break

        if not pos_doc:
            pos_emb = encode([pos_text])
            cos_scores = util.cos_sim(pos_emb, corpus_embeddings)[0]
            best_idx = torch.argmax(cos_scores)
            pos_doc = doc_ids[int(best_idx)]

        # Rankings
        Rori = retrieve_topk(q["query"])
        Rins = retrieve_topk(q["instructed_query"])
        Rrev = retrieve_topk(q["reversed_query"])

        Rori_rank = find_rank(pos_doc, Rori)
        Rins_rank = find_rank(pos_doc, Rins)
        Rrev_rank = find_rank(pos_doc, Rrev)

        # mSICR
        Iq = int((Rins_rank < Rori_rank) and (Rrev_rank > Rori_rank))

        # mWISE
        m = len(q.get("attributes", {})) or 1
        delta_ins = Rori_rank - Rins_rank
        delta_rev = Rrev_rank - Rori_rank
        reward = delta_ins / m
        penalty = -delta_rev / m
        mwise = reward + penalty

        # MDCR (soft + strict)
        attrs = q.get("attributes", {})
        mq = len(attrs)

        if mq > 0:
            pos_doc_emb = encode([pos_text])
            sim_scores = []
            for attr_name, attr_value in attrs.items():
                attr_desc = f"The document should reflect {attr_name} = {attr_value}."
                attr_emb = encode([attr_desc])
                sim = float(util.cos_sim(attr_emb, pos_doc_emb))
                sim_scores.append(sim)

            mdcr_soft = float(np.mean(sim_scores))
            threshold = max(0.45, np.mean(sim_scores) - 0.05)
            mdcr_strict = int(all(s >= threshold for s in sim_scores))
        else:
            mdcr_soft = 0.0
            mdcr_strict = 0

        results.append({
            "query_id": qid,
            "mSICR": Iq,
            "mWISE": mwise,
            "MDCR_strict": mdcr_strict,
            "MDCR_soft": mdcr_soft
        })

    except Exception as e:
        print(f"⚠ Skipped query {qid} due to error: {e}")
        continue

# =========================================================
# 8️⃣ Aggregate metrics
# =========================================================
df = pd.DataFrame(results)
metrics = {
    "model": "fine_tuned_model",
    "mSICR": df["mSICR"].mean(),
    "mWISE": df["mWISE"].mean(),
    "MDCR_strict": df["MDCR_strict"].mean(),
    "MDCR_soft": df["MDCR_soft"].mean()
}

print(f"\n📊 Fine-tuned model evaluation results:")
print(json.dumps(metrics, indent=2))


Using device: cuda
Encoding 9576 documents using fine-tuned model...


Evaluating fine-tuned model:  27%|██▋       | 2618/9596 [01:53<04:42, 24.68it/s]

⚠ Skipped query 192005 due to error: 'dict' object has no attribute 'strip'
⚠ Skipped query 192005 due to error: 'dict' object has no attribute 'strip'


Evaluating fine-tuned model:  27%|██▋       | 2626/9596 [01:53<04:03, 28.66it/s]

⚠ Skipped query 192005 due to error: 'dict' object has no attribute 'strip'
⚠ Skipped query 192005 due to error: 'dict' object has no attribute 'strip'


...

⚠ Skipped query 1042639 due to error: 'list' object has no attribute 'strip'


Evaluating fine-tuned model:  87%|████████▋ | 8360/9596 [06:00<00:54, 22.55it/s]

⚠ Skipped query 1042639 due to error: 'list' object has no attribute 'strip'


Evaluating fine-tuned model: 100%|██████████| 9596/9596 [06:54<00:00, 23.16it/s]


📊 Fine-tuned model evaluation results:
{
  "model": "fine_tuned_model",
  "mSICR": 0.07108559498956159,
  "mWISE": -1.1322199025748085,
  "MDCR_strict": 0.0,
  "MDCR_soft": 0.1375234512736619
}
